In [1]:
%%configure -f
{
"kind": "pyspark",

"conf": {
   
     "spark.master": "yarn",
    "spark.yarn.dist.archives": "/use_case/hhmatching/Library_zip/extra_lib_3.zip#lcm_zip", 
    "spark.yarn.appMasterEnv.PYSPARK_PYTHON": "./lcm_zip/bin/python",
    "spark.driver.cores": "8", 
    "spark.driver.memory": "13000m",  
    "spark.executor.memory": "13000m",
    "spark.executor.cores": "8", 
     "spark.dynamicAllocation.enabled": "true",
     "spark.dynamicAllocation.minExecutors": "3"
    
      
    }
}

### Import Libraries<a name="libs"></a>
<a href = "#toc">Back to top</a>

In [2]:
# Import required libraries
from pyspark.sql import SparkSession,SQLContext,Row, DataFrame
from pyspark import SparkContext, SparkConf, sql
import pandas as pd
import numpy as np
from pyspark.sql.types import StringType,FloatType,DoubleType,LongType,IntegerType,ArrayType
from pyspark.sql.functions import col,when,lit,abs,mean
import pyspark.sql.functions as F
from pyspark.sql.functions import unix_timestamp, from_unixtime,hour
from pyspark.sql.functions import explode,lower
import datetime

Starting Spark application


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Function Definitions<a name="funcs"></a>
<a href = "#toc">Back to top</a>

In [3]:
def dataImport(db,table):
    df = spark.sql("select * from {0}.{1}".format(db,table))
    return df

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
fc = dataImport("sbx_adhoc_ds","spark_fix_churners_202001_202101")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
fix = dataImport("stg_dwh","stg_dwh_haloph2_fix_churn_scoring")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
fc = fc.filter(F.col("churn_period")!="CHURN_PERIOD")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
fc.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

76091

In [8]:
#fc = fc.select(*fc.columns,'churn_period', from_unixtime(unix_timestamp('churn_period', 'yyyyMM')).alias('churn_date'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
#fc = fc.withColumn("churn_date",F.add_months(F.col("churn_date"),0))

# fix1.withColumn("monthDiff", F.months_between(F.col("date2"), F.col("date3"))).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
fc = fc.withColumn("flag",F.lit(1))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
fc = fc.drop('play','dataset')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
fc.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- customer_play_id: string (nullable = true)
 |-- has_maxtv: string (nullable = true)
 |-- anchor_period: string (nullable = true)
 |-- churn_period: string (nullable = true)
 |-- halo_churn_y_n: string (nullable = true)
 |-- halo_relevant: string (nullable = true)
 |-- adsl_churn_y_n: string (nullable = true)
 |-- adsl_relevant: string (nullable = true)
 |-- tv_churn_y_n: string (nullable = true)
 |-- tv_relevant: string (nullable = true)
 |-- churn_usluga: string (nullable = true)
 |-- partial_full_churn: string (nullable = true)
 |-- churn_filter: string (nullable = true)
 |-- bundle_mnths_till_contract_end: string (nullable = true)
 |-- flag: integer (nullable = false)

In [13]:
fix_all = fix.join(fc,["customer_play_id"],"left")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
fix_all = fix_all.na.fill(0,["flag"])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
fix_all.select("flag").distinct().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+
|flag|
+----+
|   1|
|   0|
+----+

In [16]:
fix_all.select([F.count(when(col(c).isNull(), c)).alias(c) for c in ["churn_period","alls_period"]]).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+-----------+
|churn_period|alls_period|
+------------+-----------+
|    15979172|          0|
+------------+-----------+

In [17]:
fix_all = fix_all.na.fill("200001",["churn_period"])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
fix_all = fix_all.select(*fix_all.columns,'churn_period', from_unixtime(unix_timestamp('churn_period', 'yyyyMM')).alias('churn_date'))

fix_all= fix_all.select(*fix_all.columns, from_unixtime(unix_timestamp('alls_period', 'yyyyMM')).alias('date'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
fix_all = fix_all.withColumn("churn_date",F.add_months(F.col("churn_date"),0))
fix_all = fix_all.withColumn("date",F.add_months(F.col("date"),0))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
fix_all.select("churn_date","date")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[churn_date: date, date: date]

In [21]:
# fix_all= fix_all.select(*fix_all.columns, from_unixtime(unix_timestamp('alls_period', 'yyyyMM')).alias('date'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
# fix_all = fix_all.withColumn("date",F.add_months(F.col("date"),0))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
fix_all1 = fix_all.withColumn("monthDiff", F.months_between(F.col("churn_date"), F.col("date")))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
month1 = fix_all1.orderBy("monthDiff",ascending = True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
month2 = month1.filter(F.col("flag") == 1 )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
month2.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1386603

In [27]:
#month3 = month2.filter((F.col("monthDiff") < 3.0) & (F.col("monthDiff") >= 0.0))
month3 = month2.filter(F.col("monthDiff") == 3.0)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
month3.select("customer_play_id","date","churn_date","monthDiff").orderBy("monthDiff",ascending = False).show(1000,False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+----------+----------+---------+
|customer_play_id|date      |churn_date|monthDiff|
+----------------+----------+----------+---------+
|28601432        |2020-02-01|2020-05-01|3.0      |
|28613084        |2020-03-01|2020-06-01|3.0      |
|28616448        |2020-04-01|2020-07-01|3.0      |
|28621831        |2020-10-01|2021-01-01|3.0      |
|28623737        |2019-10-01|2020-01-01|3.0      |
|28626877        |2019-10-01|2020-01-01|3.0      |
|28630011        |2020-10-01|2021-01-01|3.0      |
|28632394        |2019-11-01|2020-02-01|3.0      |
|28632453        |2020-01-01|2020-04-01|3.0      |
|28633236        |2020-08-01|2020-11-01|3.0      |
|28638946        |2020-04-01|2020-07-01|3.0      |
|28643147        |2019-12-01|2020-03-01|3.0      |
|28658162        |2019-11-01|2020-02-01|3.0      |
|28669026        |2020-07-01|2020-10-01|3.0      |
|28669783        |2019-10-01|2020-01-01|3.0      |
|28670771        |2020-06-01|2020-09-01|3.0      |
|28673890        |2020-08-01|20

In [34]:
month3.select("date").distinct().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+
|      date|
+----------+
|2019-11-01|
|2019-10-01|
|2020-09-01|
|2020-02-01|
|2020-08-01|
|2020-05-01|
|2020-03-01|
|2020-04-01|
|2020-07-01|
|2019-12-01|
|2020-01-01|
|2020-10-01|
|2020-06-01|
+----------+

In [35]:
month3.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

73203

In [36]:
month3.select("customer_play_id").distinct().count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

73203

In [31]:
nc_month   =  month1.filter(F.col("flag") ==0 )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
nc_month.select("customer_play_id","date","churn_date","monthDiff").orderBy("monthDiff",ascending = False).show(1000,False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+----------+----------+---------+
|customer_play_id|date      |churn_date|monthDiff|
+----------------+----------+----------+---------+
|28963878        |2018-12-01|2000-01-01|-227.0   |
|28783968        |2018-12-01|2000-01-01|-227.0   |
|28963508        |2018-12-01|2000-01-01|-227.0   |
|28689362        |2018-12-01|2000-01-01|-227.0   |
|28783873        |2018-12-01|2000-01-01|-227.0   |
|28873217        |2018-12-01|2000-01-01|-227.0   |
|28963306        |2018-12-01|2000-01-01|-227.0   |
|28642469        |2018-12-01|2000-01-01|-227.0   |
|28688709        |2018-12-01|2000-01-01|-227.0   |
|28740387        |2018-12-01|2000-01-01|-227.0   |
|28782548        |2018-12-01|2000-01-01|-227.0   |
|28825601        |2018-12-01|2000-01-01|-227.0   |
|28872576        |2018-12-01|2000-01-01|-227.0   |
|28919933        |2018-12-01|2000-01-01|-227.0   |
|28963073        |2018-12-01|2000-01-01|-227.0   |
|28625559        |2018-12-01|2000-01-01|-227.0   |
|28642399        |2018-12-01|20

In [52]:
# nc_month.select("flag").orderBy("monthDiff",ascending = True).show(1000,False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
nc_month.select("churn_date").distinct().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+
|churn_date|
+----------+
|2000-01-01|
+----------+

In [37]:
nc_month.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

15979172

In [38]:
nc_month1 = nc_month.filter((F.col("date") >= F.add_months(F.current_date(), -12)))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [39]:
nc_month1.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

6252028

In [40]:
nc_month1.select("date").distinct().show(400,False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+
|date      |
+----------+
|2021-02-01|
|2020-09-01|
|2021-01-01|
|2020-08-01|
|2020-05-01|
|2020-12-01|
|2020-04-01|
|2020-07-01|
|2020-10-01|
|2020-06-01|
+----------+

In [41]:
nc_month1.select("customer_play_id").distinct().count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

660310

In [42]:
from pyspark.sql.window import *
window_1 = Window.partitionBy(nc_month1['customer_play_id']).orderBy(F.rand())
data_1 = nc_month1.select('*', F.rank().over(window_1).alias('rank')).filter(F.col('rank') <= 1).drop('rank')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [43]:
data_1.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

660310

In [44]:
len(data_1.columns)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

170

In [45]:
len(month3.columns)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

170

In [46]:
Fix_final = data_1.union(month3.select(*data_1.columns))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [108]:
Fix_final.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

733513

In [48]:
mob = dataImport("stg_dwh","stg_dwh_haloph2_mob_churn_scoring")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [60]:

mob= mob.select(*mob.columns, from_unixtime(unix_timestamp('calendar_period', 'yyyyMM')).alias('calendar_date'))

mob = mob.withColumn("calendar_date",F.add_months(F.col("calendar_date"),0))


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [62]:
# mob.select("date").distinct().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [110]:
Fix_final.select("oib_kupac").distinct().count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

648197

In [111]:
Fix_final = Fix_final.drop("current_indicator","churn_period")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [67]:
mob_09_tele = mob

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [126]:
mob_09_tele_group = mob_09_tele.select("oib","telephone_number_id").distinct().groupby("oib").count().withColumnRenamed("count","Total_mob_connection")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [127]:
mob_09_tele_group.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+--------------------+
|        oib|Total_mob_connection|
+-----------+--------------------+
|79868864257|                   1|
|74863705332|                   3|
|26130110480|                   1|
|44670126038|                   1|
|07889126056|                   2|
|88822578657|                   3|
|54263286428|                   3|
|66760224102|                   5|
|48297222074|                   2|
|34461549577|                   1|
|39215637131|                   3|
|59063504423|                   1|
|07603966499|                   1|
|29765152677|                   1|
|35587635508|                   1|
|49786241566|                   2|
|39104946387|                   2|
|45946801252|                   1|
|18296915436|                   1|
|45807444271|                   3|
+-----------+--------------------+
only showing top 20 rows

In [68]:
L1 = [
    
    'QTY_COMPLAINT_PMIN2',
'QTY_COMPLAINT_PMIN3',
'QTY_COMPLAINT_PMIN4',
'RATE_TMHR_CALLS_PMIN2',
'RATE_BONBON_CALLS_PMIN2',
'RATE_VIP_CALLS_PMIN2',
'RATE_TELE2_CALLS_PMIN2',
'RATE_TOMATO_CALLS_PMIN2',
'RATE_INTRNTIONAL_CALLS_PMIN2',
'RATE_FIXED_CALLS_PMIN2',
'RATE_VAS_CALLS_PMIN2',
'RATE_ROAMING_CALLS_PMIN2',
'RATE_TMHR_CALLS_PMIN3',
'RATE_BONBON_CALLS_PMIN3',
'RATE_VIP_CALLS_PMIN3',
'RATE_TELE2_CALLS_PMIN3',
'RATE_TOMATO_CALLS_PMIN3',
'RATE_INTRNTIONAL_CALLS_PMIN3',
'RATE_FIXED_CALLS_PMIN3',
'RATE_VAS_CALLS_PMIN3',
'RATE_ROAMING_CALLS_PMIN3',
'RATE_TMHR_CALLS_PMIN4',
'RATE_BONBON_CALLS_PMIN4',
'RATE_VIP_CALLS_PMIN4',
'RATE_TELE2_CALLS_PMIN4',
'RATE_TOMATO_CALLS_PMIN4',
'RATE_INTRNTIONAL_CALLS_PMIN4',
'RATE_FIXED_CALLS_PMIN4',
'RATE_VAS_CALLS_PMIN4',
'RATE_ROAMING_CALLS_PMIN4',
'RATE_TMHR_MIN_PMIN2',
'RATE_BONBON_MIN_PMIN2',
'RATE_VIP_MIN_PMIN2',
'RATE_TELE2_MIN_PMIN2',
'RATE_TOMATO_MIN_PMIN2',
'RATE_INTRNTIONAL_MIN_PMIN2',
'RATE_FIXED_MIN_PMIN2',
'RATE_VAS_MIN_PMIN2',
'RATE_ROAMING_MIN_PMIN2',
'RATE_TMHR_MIN_PMIN3',
'RATE_BONBON_MIN_PMIN3',
'RATE_VIP_MIN_PMIN3',
'RATE_TELE2_MIN_PMIN3',
'RATE_TOMATO_MIN_PMIN3',
'RATE_INTRNTIONAL_MIN_PMIN3',
'RATE_FIXED_MIN_PMIN3',
'RATE_VAS_MIN_PMIN3',
'RATE_ROAMING_MIN_PMIN3',
'RATE_TMHR_MIN_PMIN4',
'RATE_BONBON_MIN_PMIN4',
'RATE_VIP_MIN_PMIN4',
'RATE_TELE2_MIN_PMIN4',
'RATE_TOMATO_MIN_PMIN4',
'RATE_INTRNTIONAL_MIN_PMIN4',
'RATE_FIXED_MIN_PMIN4',
'RATE_VAS_MIN_PMIN4',
'RATE_ROAMING_MIN_PMIN4'
]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [69]:
L2 = [
    
'QTY_FIX_INST_IN_HH_PMIN2',
'QTY_FIX_INST_IN_HH_PMIN3',
'QTY_FIX_INST_IN_HH_PMIN4',
'QTY_MOB_IN_HH_PMIN2',
'QTY_MOB_IN_HH_PMIN3',
'QTY_MOB_IN_HH_PMIN4'

]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [73]:
mob_09_tele_max = mob_09_tele.select("oib","calendar_date",*L1,*L2)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [74]:
mob_09_tele_max = mob_09_tele_max.withColumn("QTY_COMPLAINT_PMIN2",F.col("QTY_COMPLAINT_PMIN2").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("QTY_COMPLAINT_PMIN3",F.col("QTY_COMPLAINT_PMIN3").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("QTY_COMPLAINT_PMIN4",F.col("QTY_COMPLAINT_PMIN4").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_TMHR_CALLS_PMIN2",F.col("RATE_TMHR_CALLS_PMIN2").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_BONBON_CALLS_PMIN2",F.col("RATE_BONBON_CALLS_PMIN2").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_VIP_CALLS_PMIN2",F.col("RATE_VIP_CALLS_PMIN2").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_TELE2_CALLS_PMIN2",F.col("RATE_TELE2_CALLS_PMIN2").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_TOMATO_CALLS_PMIN2",F.col("RATE_TOMATO_CALLS_PMIN2").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_INTRNTIONAL_CALLS_PMIN2",F.col("RATE_INTRNTIONAL_CALLS_PMIN2").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_FIXED_CALLS_PMIN2",F.col("RATE_FIXED_CALLS_PMIN2").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_VAS_CALLS_PMIN2",F.col("RATE_VAS_CALLS_PMIN2").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_ROAMING_CALLS_PMIN2",F.col("RATE_ROAMING_CALLS_PMIN2").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_TMHR_CALLS_PMIN3",F.col("RATE_TMHR_CALLS_PMIN3").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_BONBON_CALLS_PMIN3",F.col("RATE_BONBON_CALLS_PMIN3").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_VIP_CALLS_PMIN3",F.col("RATE_VIP_CALLS_PMIN3").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_TELE2_CALLS_PMIN3",F.col("RATE_TELE2_CALLS_PMIN3").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_TOMATO_CALLS_PMIN3",F.col("RATE_TOMATO_CALLS_PMIN3").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_INTRNTIONAL_CALLS_PMIN3",F.col("RATE_INTRNTIONAL_CALLS_PMIN3").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_FIXED_CALLS_PMIN3",F.col("RATE_FIXED_CALLS_PMIN3").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_VAS_CALLS_PMIN3",F.col("RATE_VAS_CALLS_PMIN3").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_ROAMING_CALLS_PMIN3",F.col("RATE_ROAMING_CALLS_PMIN3").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_TMHR_CALLS_PMIN4",F.col("RATE_TMHR_CALLS_PMIN4").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_BONBON_CALLS_PMIN4",F.col("RATE_BONBON_CALLS_PMIN4").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_VIP_CALLS_PMIN4",F.col("RATE_VIP_CALLS_PMIN4").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_TELE2_CALLS_PMIN4",F.col("RATE_TELE2_CALLS_PMIN4").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_TOMATO_CALLS_PMIN4",F.col("RATE_TOMATO_CALLS_PMIN4").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_INTRNTIONAL_CALLS_PMIN4",F.col("RATE_INTRNTIONAL_CALLS_PMIN4").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_FIXED_CALLS_PMIN4",F.col("RATE_FIXED_CALLS_PMIN4").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_VAS_CALLS_PMIN4",F.col("RATE_VAS_CALLS_PMIN4").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_ROAMING_CALLS_PMIN4",F.col("RATE_ROAMING_CALLS_PMIN4").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_TMHR_MIN_PMIN2",F.col("RATE_TMHR_MIN_PMIN2").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_BONBON_MIN_PMIN2",F.col("RATE_BONBON_MIN_PMIN2").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_VIP_MIN_PMIN2",F.col("RATE_VIP_MIN_PMIN2").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_TELE2_MIN_PMIN2",F.col("RATE_TELE2_MIN_PMIN2").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_TOMATO_MIN_PMIN2",F.col("RATE_TOMATO_MIN_PMIN2").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_INTRNTIONAL_MIN_PMIN2",F.col("RATE_INTRNTIONAL_MIN_PMIN2").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_FIXED_MIN_PMIN2",F.col("RATE_FIXED_MIN_PMIN2").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_VAS_MIN_PMIN2",F.col("RATE_VAS_MIN_PMIN2").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_ROAMING_MIN_PMIN2",F.col("RATE_ROAMING_MIN_PMIN2").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_TMHR_MIN_PMIN3",F.col("RATE_TMHR_MIN_PMIN3").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_BONBON_MIN_PMIN3",F.col("RATE_BONBON_MIN_PMIN3").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_VIP_MIN_PMIN3",F.col("RATE_VIP_MIN_PMIN3").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_TELE2_MIN_PMIN3",F.col("RATE_TELE2_MIN_PMIN3").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_TOMATO_MIN_PMIN3",F.col("RATE_TOMATO_MIN_PMIN3").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_INTRNTIONAL_MIN_PMIN3",F.col("RATE_INTRNTIONAL_MIN_PMIN3").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_FIXED_MIN_PMIN3",F.col("RATE_FIXED_MIN_PMIN3").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_VAS_MIN_PMIN3",F.col("RATE_VAS_MIN_PMIN3").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_ROAMING_MIN_PMIN3",F.col("RATE_ROAMING_MIN_PMIN3").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_TMHR_MIN_PMIN4",F.col("RATE_TMHR_MIN_PMIN4").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_BONBON_MIN_PMIN4",F.col("RATE_BONBON_MIN_PMIN4").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_VIP_MIN_PMIN4",F.col("RATE_VIP_MIN_PMIN4").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_TELE2_MIN_PMIN4",F.col("RATE_TELE2_MIN_PMIN4").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_TOMATO_MIN_PMIN4",F.col("RATE_TOMATO_MIN_PMIN4").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_INTRNTIONAL_MIN_PMIN4",F.col("RATE_INTRNTIONAL_MIN_PMIN4").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_FIXED_MIN_PMIN4",F.col("RATE_FIXED_MIN_PMIN4").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_VAS_MIN_PMIN4",F.col("RATE_VAS_MIN_PMIN4").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_ROAMING_MIN_PMIN4",F.col("RATE_ROAMING_MIN_PMIN4").cast(IntegerType()))
#mob_09_tele_max = mob_09_tele_max.withColumn("MONTHS_TILL_CONTRACT_END_PMIN2",F.col("MONTHS_TILL_CONTRACT_END_PMIN2").cast(IntegerType()))


mob_09_tele_max = mob_09_tele_max.withColumn("QTY_FIX_INST_IN_HH_PMIN2",F.col("QTY_FIX_INST_IN_HH_PMIN2").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("QTY_FIX_INST_IN_HH_PMIN3",F.col("QTY_FIX_INST_IN_HH_PMIN3").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("QTY_FIX_INST_IN_HH_PMIN4",F.col("QTY_FIX_INST_IN_HH_PMIN4").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("QTY_MOB_IN_HH_PMIN2",F.col("QTY_MOB_IN_HH_PMIN2").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("QTY_MOB_IN_HH_PMIN3",F.col("QTY_MOB_IN_HH_PMIN3").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("QTY_MOB_IN_HH_PMIN4",F.col("QTY_MOB_IN_HH_PMIN4").cast(IntegerType()))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [75]:
mob_09_tele_max = mob_09_tele_max.groupBy("oib","calendar_date").agg(
    F.sum("QTY_FIX_INST_IN_HH_PMIN2").alias("QTY_FIX_INST_IN_HH_PMIN2"),
    F.sum("QTY_FIX_INST_IN_HH_PMIN3").alias("QTY_FIX_INST_IN_HH_PMIN3"),
    F.sum("QTY_FIX_INST_IN_HH_PMIN4").alias("QTY_FIX_INST_IN_HH_PMIN4"),
    F.sum("QTY_MOB_IN_HH_PMIN2").alias("QTY_MOB_IN_HH_PMIN2"),
    F.sum("QTY_MOB_IN_HH_PMIN3").alias("QTY_MOB_IN_HH_PMIN3"),
    F.sum("QTY_MOB_IN_HH_PMIN4").alias("QTY_MOB_IN_HH_PMIN4"),
    
# F.min("MONTHS_TILL_CONTRACT_END_PMIN2").alias("MONTHS_TILL_CONTRACT_END_PMIN2"),
F.max("QTY_COMPLAINT_PMIN2").alias("QTY_COMPLAINT_PMIN2"),
F.max("QTY_COMPLAINT_PMIN3").alias("QTY_COMPLAINT_PMIN3"),
F.max("QTY_COMPLAINT_PMIN4").alias("QTY_COMPLAINT_PMIN4"),
F.max("RATE_TMHR_CALLS_PMIN2").alias("RATE_TMHR_CALLS_PMIN2"),
F.max("RATE_BONBON_CALLS_PMIN2").alias("RATE_BONBON_CALLS_PMIN2"),
F.max("RATE_VIP_CALLS_PMIN2").alias("RATE_VIP_CALLS_PMIN2"),
F.max("RATE_TELE2_CALLS_PMIN2").alias("RATE_TELE2_CALLS_PMIN2"),
F.max("RATE_TOMATO_CALLS_PMIN2").alias("RATE_TOMATO_CALLS_PMIN2"),
F.max("RATE_INTRNTIONAL_CALLS_PMIN2").alias("RATE_INTRNTIONAL_CALLS_PMIN2"),
F.max("RATE_FIXED_CALLS_PMIN2").alias("RATE_FIXED_CALLS_PMIN2"),
F.max("RATE_VAS_CALLS_PMIN2").alias("RATE_VAS_CALLS_PMIN2"),
F.max("RATE_ROAMING_CALLS_PMIN2").alias("RATE_ROAMING_CALLS_PMIN2"),
F.max("RATE_TMHR_CALLS_PMIN3").alias("RATE_TMHR_CALLS_PMIN3"),
F.max("RATE_BONBON_CALLS_PMIN3").alias("RATE_BONBON_CALLS_PMIN3"),
F.max("RATE_VIP_CALLS_PMIN3").alias("RATE_VIP_CALLS_PMIN3"),
F.max("RATE_TELE2_CALLS_PMIN3").alias("RATE_TELE2_CALLS_PMIN3"),
F.max("RATE_TOMATO_CALLS_PMIN3").alias("RATE_TOMATO_CALLS_PMIN3"),
F.max("RATE_INTRNTIONAL_CALLS_PMIN3").alias("RATE_INTRNTIONAL_CALLS_PMIN3"),
F.max("RATE_FIXED_CALLS_PMIN3").alias("RATE_FIXED_CALLS_PMIN3"),
F.max("RATE_VAS_CALLS_PMIN3").alias("RATE_VAS_CALLS_PMIN3"),
F.max("RATE_ROAMING_CALLS_PMIN3").alias("RATE_ROAMING_CALLS_PMIN3"),
F.max("RATE_TMHR_CALLS_PMIN4").alias("RATE_TMHR_CALLS_PMIN4"),
F.max("RATE_BONBON_CALLS_PMIN4").alias("RATE_BONBON_CALLS_PMIN4"),
F.max("RATE_VIP_CALLS_PMIN4").alias("RATE_VIP_CALLS_PMIN4"),
F.max("RATE_TELE2_CALLS_PMIN4").alias("RATE_TELE2_CALLS_PMIN4"),
F.max("RATE_TOMATO_CALLS_PMIN4").alias("RATE_TOMATO_CALLS_PMIN4"),
F.max("RATE_INTRNTIONAL_CALLS_PMIN4").alias("RATE_INTRNTIONAL_CALLS_PMIN4"),
F.max("RATE_FIXED_CALLS_PMIN4").alias("RATE_FIXED_CALLS_PMIN4"),
F.max("RATE_VAS_CALLS_PMIN4").alias("RATE_VAS_CALLS_PMIN4"),
F.max("RATE_ROAMING_CALLS_PMIN4").alias("RATE_ROAMING_CALLS_PMIN4"),
F.max("RATE_TMHR_MIN_PMIN2").alias("RATE_TMHR_MIN_PMIN2"),
F.max("RATE_BONBON_MIN_PMIN2").alias("RATE_BONBON_MIN_PMIN2"),
F.max("RATE_VIP_MIN_PMIN2").alias("RATE_VIP_MIN_PMIN2"),
F.max("RATE_TELE2_MIN_PMIN2").alias("RATE_TELE2_MIN_PMIN2"),
F.max("RATE_TOMATO_MIN_PMIN2").alias("RATE_TOMATO_MIN_PMIN2"),
F.max("RATE_INTRNTIONAL_MIN_PMIN2").alias("RATE_INTRNTIONAL_MIN_PMIN2"),
F.max("RATE_FIXED_MIN_PMIN2").alias("RATE_FIXED_MIN_PMIN2"),
F.max("RATE_VAS_MIN_PMIN2").alias("RATE_VAS_MIN_PMIN2"),
F.max("RATE_ROAMING_MIN_PMIN2").alias("RATE_ROAMING_MIN_PMIN2"),
F.max("RATE_TMHR_MIN_PMIN3").alias("RATE_TMHR_MIN_PMIN3"),
F.max("RATE_BONBON_MIN_PMIN3").alias("RATE_BONBON_MIN_PMIN3"),
F.max("RATE_VIP_MIN_PMIN3").alias("RATE_VIP_MIN_PMIN3"),
F.max("RATE_TELE2_MIN_PMIN3").alias("RATE_TELE2_MIN_PMIN3"),
F.max("RATE_TOMATO_MIN_PMIN3").alias("RATE_TOMATO_MIN_PMIN3"),
F.max("RATE_INTRNTIONAL_MIN_PMIN3").alias("RATE_INTRNTIONAL_MIN_PMIN3"),
F.max("RATE_FIXED_MIN_PMIN3").alias("RATE_FIXED_MIN_PMIN3"),
F.max("RATE_VAS_MIN_PMIN3").alias("RATE_VAS_MIN_PMIN3"),
F.max("RATE_ROAMING_MIN_PMIN3").alias("RATE_ROAMING_MIN_PMIN3"),
F.max("RATE_TMHR_MIN_PMIN4").alias("RATE_TMHR_MIN_PMIN4"),
F.max("RATE_BONBON_MIN_PMIN4").alias("RATE_BONBON_MIN_PMIN4"),
F.max("RATE_VIP_MIN_PMIN4").alias("RATE_VIP_MIN_PMIN4"),
F.max("RATE_TELE2_MIN_PMIN4").alias("RATE_TELE2_MIN_PMIN4"),
F.max("RATE_TOMATO_MIN_PMIN4").alias("RATE_TOMATO_MIN_PMIN4"),
F.max("RATE_INTRNTIONAL_MIN_PMIN4").alias("RATE_INTRNTIONAL_MIN_PMIN4"),
F.max("RATE_FIXED_MIN_PMIN4").alias("RATE_FIXED_MIN_PMIN4"),
F.max("RATE_VAS_MIN_PMIN4").alias("RATE_VAS_MIN_PMIN4"),
F.max("RATE_ROAMING_MIN_PMIN4").alias("RATE_ROAMING_MIN_PMIN4"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [76]:
mob_09_tele_max.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

7318347

In [77]:
L3 = [
    
    'VOICE_ALL_CALLS_PMIN2',
'VOICE_ALL_CALLS_PMIN3',
'VOICE_ALL_CALLS_PMIN4',
'VOICE_TMHR_CALLS_PMIN2',
'VOICE_TMHR_CALLS_PMIN3',
'VOICE_TMHR_CALLS_PMIN4',
'VOICE_BONBON_CALLS_PMIN2',
'VOICE_BONBON_CALLS_PMIN3',
'VOICE_BONBON_CALLS_PMIN4',
'VOICE_VIP_CALLS_PMIN2',
'VOICE_VIP_CALLS_PMIN3',
'VOICE_VIP_CALLS_PMIN4',
'VOICE_TELE2_CALLS_PMIN2',
'VOICE_TELE2_CALLS_PMIN3',
'VOICE_TELE2_CALLS_PMIN4',
'VOICE_TOMATO_CALLS_PMIN2',
'VOICE_TOMATO_CALLS_PMIN3',
'VOICE_TOMATO_CALLS_PMIN4',
'VOICE_INTRNTIONAL_CALLS_PMIN2',
'VOICE_INTRNTIONAL_CALLS_PMIN3',
'VOICE_INTRNTIONAL_CALLS_PMIN4',
'VOICE_FIXED_CALLS_MIN2',
'VOICE_FIXED_CALLS_MIN3',
'VOICE_FIXED_CALLS_MIN4',
'VOICE_VAS_CALLS_PMIN2',
'VOICE_VAS_CALLS_PMIN3',
'VOICE_VAS_CALLS_PMIN4',
'VOICE_ROAMING_CALLS_PMIN2',
'VOICE_ROAMING_CALLS_PMIN3',
'VOICE_ROAMING_CALLS_PMIN4',
'VOICE_ALL_MIN_PMIN2',
'VOICE_ALL_MIN_PMIN3',
'VOICE_ALL_MIN_PMIN4',
'VOICE_TMHR_MIN_PMIN2',
'VOICE_TMHR_MIN_PMIN3',
'VOICE_TMHR_MIN_PMIN4',
# 'VOICE_BONBON_MIN_PMIN_2',
#'VOICE_BONBON_MIN_PMIN_3',
#'VOICE_BONBON_MIN_PMIN_4',
'VOICE_VIP_MIN_PMIN2',
'VOICE_VIP_MIN_PMIN3',
'VOICE_VIP_MIN_PMIN4',
'VOICE_TELE2_MIN_PMIN2',
'VOICE_TELE2_MIN_PMIN3',
'VOICE_TELE2_MIN_PMIN4',
'VOICE_TOMATO_MIN_PMIN2',
'VOICE_TOMATO_MIN_PMIN3',
'VOICE_TOMATO_MIN_PMIN4',
'VOICE_INTERNATIONAL_MIN_PMIN2',
'VOICE_INTERNATIONAL_MIN_PMIN3',
'VOICE_INTERNATIONAL_MIN_PMIN4',
'VOICE_FIXED_MIN_PMIN2',
'VOICE_FIXED_MIN_PMIN3',
'VOICE_FIXED_MIN_PMIN4',
'VOICE_VAS_MIN_PMIN2',
'VOICE_VAS_MIN_PMIN3',
'VOICE_VAS_MIN_PMIN4',
'VOICE_ROAMING_MIN_PMIN2',
'VOICE_ROAMING_MIN_PMIN3',
'VOICE_ROAMING_MIN_PMIN4',
'SMS_ALL_PMIN2',
'SMS_ALL_PMIN3',
'SMS_ALL_PMIN4',
'SMS_TMHR_PMIN2',
'SMS_TMHR_PMIN3',
'SMS_TMHR_PMIN4',
'SMS_BONBON_PMIN2',
'SMS_BONBON_PMIN3',
'SMS_BONBON_PMIN4',
'SMS_VIP_PMIN2',
'SMS_VIP_PMIN3',
'SMS_VIP_PMIN4',
'SMS_TELE2_PMIN2',
'SMS_TELE2_PMIN3',
'SMS_TELE2_PMIN4',
'SMS_TOMATO_PMIN2',
'SMS_TOMATO_PMIN3',
'SMS_TOMATO_PMIN4',
'SMS_INTERNATIONAL_PMIN2',
'SMS_INTERNATIONAL_PMIN3',
'SMS_INTERNATIONAL_PMIN4',
'SMS_VAS_PMIN2',
'SMS_VAS_PMIN3',
'SMS_VAS_PMIN4',
'SMS_ROAMING_PMIN2',
'SMS_ROAMING_PMIN3',
'SMS_ROAMING_PMIN4',
'DATA_NATIONAL_VOLUME_MB_PMIN2',
'DATA_NATIONAL_VOLUME_MB_PMIN3',
'DATA_NATIONAL_VOLUME_MB_PMIN4',
'DATA_NATIONAL_SESSIONS_PMIN2',
'DATA_NATIONAL_SESSIONS_PMIN3',
'DATA_NATIONAL_SESSIONS_PMIN4',
'DATA_ROAMING_VOLUME_MB_PMIN2',
'DATA_ROAMING_VOLUME_MB_PMIN3',
'DATA_ROAMING_VOLUME_MB_PMIN4',
'DATA_ROAMING_SESSIONS_PMIN2',
'DATA_ROAMING_SESSIONS_PMIN3',
'DATA_ROAMING_SESSIONS_PMIN4',
'INT_CALL_IN_ADMIN_ACT_PMIN2',
'INT_CALL_IN_ADMIN_ACT_PMIN3',
'INT_CALL_IN_ADMIN_ACT_PMIN4',
'INT_CALL_IN_HRC_PMIN2',
'INT_CALL_IN_HRC_PMIN3',
'INT_CALL_IN_HRC_PMIN4',
'INT_CALL_IN_ICCA_TIM_PMIN2',
'INT_CALL_IN_ICCA_TIM_PMIN3',
'INT_CALL_IN_ICCA_TIM_PMIN4',
'INT_CALL_IN_INFO_PMIN2',
'INT_CALL_IN_INFO_PMIN3',
'INT_CALL_IN_INFO_PMIN4',
'INT_CALL_IN_INTERACTION_PMIN2',
'INT_CALL_IN_INTERACTION_PMIN3',
'INT_CALL_IN_INTERACTION_PMIN4',
'INT_CALL_IN_USLUGE_KAMP_PMIN2',
'INT_CALL_IN_USLUGE_KAMP_PMIN3',
'INT_CALL_IN_USLUGE_KAMP_PMIN4',
'INT_CALL_IN_OSTALO_PMIN2',
'INT_CALL_IN_OSTALO_PMIN3',
'INT_CALL_IN_OSTALO_PMIN4',
'INT_CALL_IN_RACUNI_PMIN2',
'INT_CALL_IN_RACUNI_PMIN3',
'INT_CALL_IN_RACUNI_PMIN4',
'INT_CALL_IN_POS_KOR_PMIN2',
'INT_CALL_IN_POS_KOR_PMIN3',
'INT_CALL_IN_POS_KOR_PMIN4',
'INT_CALL_IN_PRIGOVOR_PMIN2',
'INT_CALL_IN_PRIGOVOR_PMIN3',
'INT_CALL_IN_PRIGOVOR_PMIN4',
'INT_CALL_IN_UGOVOR_PMIN2',
'INT_CALL_IN_UGOVOR_PMIN3',
'INT_CALL_IN_UGOVOR_PMIN4',
'INT_CALL_IN_SAVEDESK_PMIN2',
'INT_CALL_IN_SAVEDESK_PMIN3',
'INT_CALL_IN_SAVEDESK_PMIN4',
'INT_CALL_IN_TEHNICKA_PMIN2',
'INT_CALL_IN_TEHNICKA_PMIN3',
'INT_CALL_IN_TEHNICKA_PMIN4',
'INT_CALL_IN_TELESALES_PMIN2',
'INT_CALL_IN_TELESALES_PMIN3',
'INT_CALL_IN_TELESALES_PMIN4',
'INT_CALL_IN_TRIM_TIM_PMIN2',
'INT_CALL_IN_TRIM_TIM_PMIN3',
'INT_CALL_IN_TRIM_TIM_PMIN4',
'INT_CALL_IN_X7_PMIN2',
'INT_CALL_IN_X7_PMIN3',
'INT_CALL_IN_X7_PMIN4',
'QTY_DAYS_INTERACTING_PMIN2',
'QTY_DAYS_INTERACTING_PMIN3',
'QTY_DAYS_INTERACTING_PMIN4',
'INVOICED_PMIN2',
'INVOICED_PMIN3',
'INVOICED_PMIN4',
'REVENUE_PMIN2',
'REVENUE_PMIN3',
'REVENUE_PMIN4',
'ROAMING_AMOUNT_PMIN2',
'ROAMING_AMOUNT_PMIN3',
'ROAMING_AMOUNT_PMIN4',
'REVENUE_SUBSCRIPTION_PMIN2',
'REVENUE_SUBSCRIPTION_PMIN3',
'REVENUE_SUBSCRIPTION_PMIN4',
'REVENUE_VOICE_PMIN2',
'REVENUE_VOICE_PMIN3',
'REVENUE_VOICE_PMIN4',
'REVENUE_SMS_PMIN2',
'REVENUE_SMS_PMIN3',
'REVENUE_SMS_PMIN4',
'REVENUE_DATA_NATIONAL_PMIN2',
'REVENUE_DATA_NATIONAL_PMIN3',
'REVENUE_DATA_NATIONAL_PMIN4',
'REVENUE_DATA_ROAMING_PMIN2',
'REVENUE_DATA_ROAMING_PMIN3',
'REVENUE_DATA_ROAMING_PMIN4',
'REVENUE_VOICE_VAS_PMIN2',
'REVENUE_VOICE_VAS_PMIN3',
'REVENUE_VOICE_VAS_PMIN4',
'REVENUE_SMS_VAS_PMIN2',
'REVENUE_SMS_VAS_PMIN3',
'REVENUE_SMS_VAS_PMIN4',
'REVENUE_GSM_LATE_FEE_PMIN2',
'REVENUE_GSM_LATE_FEE_PMIN3',
'REVENUE_GSM_LATE_FEE_PMIN4',
'REVENUE_GSM_OTHER_PMIN2',
'REVENUE_GSM_OTHER_PMIN3',
'REVENUE_GSM_OTHER_PMIN4',
'COST_VOICE_ONNET_PMIN2',
'COST_VOICE_ONNET_PMIN3',
'COST_VOICE_ONNET_PMIN4',
'COST_VOICE_OFFNET_PMIN2',
'COST_VOICE_OFFNET_PMIN3',
'COST_VOICE_OFFNET_PMIN4',
'COST_VOICE_VIP_PMIN2',
'COST_VOICE_VIP_PMIN3',
'COST_VOICE_VIP_PMIN4',
'COST_VOICE_TELE2_PMIN2',
'COST_VOICE_TELE2_PMIN3',
'COST_VOICE_TELE2_PMIN4',
'COST_VOICE_TOMATO_PMIN2',
'COST_VOICE_TOMATO_PMIN3',
'COST_VOICE_TOMATO_PMIN4',
'COST_VOICE_BONBON_PMIN2',
'COST_VOICE_BONBON_PMIN3',
'COST_VOICE_BONBON_PMIN4',
'COST_VOICE_FIX_PMIN2',
'COST_VOICE_FIX_PMIN3',
'COST_VOICE_FIX_PMIN4',
'COST_VOICE_INT_PMIN2',
'COST_VOICE_INT_PMIN3',
'COST_VOICE_INT_PMIN4',
'COST_VOICE_ROAMING_PMIN2',
'COST_VOICE_ROAMING_PMIN3',
'COST_VOICE_ROAMING_PMIN4',
'COST_VOICE_TOTAL_PMIN2',
'COST_VOICE_TOTAL_PMIN3',
'COST_VOICE_TOTAL_PMIN4',
'COST_VOICE_VAS_PMIN2',
'COST_VOICE_VAS_PMIN3',
'COST_VOICE_VAS_PMIN4',
'COST_SMS_ONNET_PMIN2',
'COST_SMS_ONNET_PMIN3',
'COST_SMS_ONNET_PMIN4',
'COST_SMS_OFFNET_PMIN2',
'COST_SMS_OFFNET_PMIN3',
'COST_SMS_OFFNET_PMIN4',
'COST_SMS_VIP_PMIN2',
'COST_SMS_VIP_PMIN3',
'COST_SMS_VIP_PMIN4',
'COST_SMS_TELE2_PMIN2',
'COST_SMS_TELE2_PMIN3',
'COST_SMS_TELE2_PMIN4',
'COST_SMS_TOMATO_PMIN2',
'COST_SMS_TOMATO_PMIN3',
'COST_SMS_TOMATO_PMIN4',
'COST_SMS_BONBON_PMIN2',
'COST_SMS_BONBON_PMIN3',
'COST_SMS_BONBON_PMIN4',
'COST_SMS_TOTAL_PMIN2',
'COST_SMS_TOTAL_PMIN3',
'COST_SMS_TOTAL_PMIN4',
'COST_SMS_ROAMING_PMIN2',
'COST_SMS_ROAMING_PMIN3',
'COST_SMS_ROAMING_PMIN4',
'COST_MMS_TOTAL_PMIN2',
'COST_MMS_TOTAL_PMIN3',
'COST_MMS_TOTAL_PMIN4',
'COST_DATA_PMIN2',
'COST_DATA_PMIN3',
'COST_DATA_PMIN4',
'COST_DATA_NATIONAL_PMIN2',
'COST_DATA_NATIONAL_PMIN3',
'COST_DATA_NATIONAL_PMIN4'

]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [78]:
mob_09_tele_avg = mob_09_tele.select("oib","calendar_date",*L3)
        

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [79]:
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_ALL_CALLS_PMIN2",F.col("VOICE_ALL_CALLS_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_ALL_CALLS_PMIN3",F.col("VOICE_ALL_CALLS_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_ALL_CALLS_PMIN4",F.col("VOICE_ALL_CALLS_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_TMHR_CALLS_PMIN2",F.col("VOICE_TMHR_CALLS_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_TMHR_CALLS_PMIN3",F.col("VOICE_TMHR_CALLS_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_TMHR_CALLS_PMIN4",F.col("VOICE_TMHR_CALLS_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_BONBON_CALLS_PMIN2",F.col("VOICE_BONBON_CALLS_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_BONBON_CALLS_PMIN3",F.col("VOICE_BONBON_CALLS_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_BONBON_CALLS_PMIN4",F.col("VOICE_BONBON_CALLS_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_VIP_CALLS_PMIN2",F.col("VOICE_VIP_CALLS_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_VIP_CALLS_PMIN3",F.col("VOICE_VIP_CALLS_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_VIP_CALLS_PMIN4",F.col("VOICE_VIP_CALLS_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_TELE2_CALLS_PMIN2",F.col("VOICE_TELE2_CALLS_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_TELE2_CALLS_PMIN3",F.col("VOICE_TELE2_CALLS_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_TELE2_CALLS_PMIN4",F.col("VOICE_TELE2_CALLS_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_TOMATO_CALLS_PMIN2",F.col("VOICE_TOMATO_CALLS_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_TOMATO_CALLS_PMIN3",F.col("VOICE_TOMATO_CALLS_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_TOMATO_CALLS_PMIN4",F.col("VOICE_TOMATO_CALLS_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_INTRNTIONAL_CALLS_PMIN2",F.col("VOICE_INTRNTIONAL_CALLS_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_INTRNTIONAL_CALLS_PMIN3",F.col("VOICE_INTRNTIONAL_CALLS_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_INTRNTIONAL_CALLS_PMIN4",F.col("VOICE_INTRNTIONAL_CALLS_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_FIXED_CALLS_MIN2",F.col("VOICE_FIXED_CALLS_MIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_FIXED_CALLS_MIN3",F.col("VOICE_FIXED_CALLS_MIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_FIXED_CALLS_MIN4",F.col("VOICE_FIXED_CALLS_MIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_VAS_CALLS_PMIN2",F.col("VOICE_VAS_CALLS_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_VAS_CALLS_PMIN3",F.col("VOICE_VAS_CALLS_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_VAS_CALLS_PMIN4",F.col("VOICE_VAS_CALLS_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_ROAMING_CALLS_PMIN2",F.col("VOICE_ROAMING_CALLS_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_ROAMING_CALLS_PMIN3",F.col("VOICE_ROAMING_CALLS_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_ROAMING_CALLS_PMIN4",F.col("VOICE_ROAMING_CALLS_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_ALL_MIN_PMIN2",F.col("VOICE_ALL_MIN_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_ALL_MIN_PMIN3",F.col("VOICE_ALL_MIN_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_ALL_MIN_PMIN4",F.col("VOICE_ALL_MIN_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_TMHR_MIN_PMIN2",F.col("VOICE_TMHR_MIN_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_TMHR_MIN_PMIN3",F.col("VOICE_TMHR_MIN_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_TMHR_MIN_PMIN4",F.col("VOICE_TMHR_MIN_PMIN4").cast(IntegerType()))
# mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_BONBON_MIN_PMIN_2",F.col("VOICE_BONBON_MIN_PMIN_2").cast(IntegerType()))
# mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_BONBON_MIN_PMIN_3",F.col("VOICE_BONBON_MIN_PMIN_3").cast(IntegerType()))
# mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_BONBON_MIN_PMIN_4",F.col("VOICE_BONBON_MIN_PMIN_4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_VIP_MIN_PMIN2",F.col("VOICE_VIP_MIN_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_VIP_MIN_PMIN3",F.col("VOICE_VIP_MIN_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_VIP_MIN_PMIN4",F.col("VOICE_VIP_MIN_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_TELE2_MIN_PMIN2",F.col("VOICE_TELE2_MIN_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_TELE2_MIN_PMIN3",F.col("VOICE_TELE2_MIN_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_TELE2_MIN_PMIN4",F.col("VOICE_TELE2_MIN_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_TOMATO_MIN_PMIN2",F.col("VOICE_TOMATO_MIN_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_TOMATO_MIN_PMIN3",F.col("VOICE_TOMATO_MIN_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_TOMATO_MIN_PMIN4",F.col("VOICE_TOMATO_MIN_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_INTERNATIONAL_MIN_PMIN2",F.col("VOICE_INTERNATIONAL_MIN_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_INTERNATIONAL_MIN_PMIN3",F.col("VOICE_INTERNATIONAL_MIN_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_INTERNATIONAL_MIN_PMIN4",F.col("VOICE_INTERNATIONAL_MIN_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_FIXED_MIN_PMIN2",F.col("VOICE_FIXED_MIN_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_FIXED_MIN_PMIN3",F.col("VOICE_FIXED_MIN_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_FIXED_MIN_PMIN4",F.col("VOICE_FIXED_MIN_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_VAS_MIN_PMIN2",F.col("VOICE_VAS_MIN_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_VAS_MIN_PMIN3",F.col("VOICE_VAS_MIN_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_VAS_MIN_PMIN4",F.col("VOICE_VAS_MIN_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_ROAMING_MIN_PMIN2",F.col("VOICE_ROAMING_MIN_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_ROAMING_MIN_PMIN3",F.col("VOICE_ROAMING_MIN_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_ROAMING_MIN_PMIN4",F.col("VOICE_ROAMING_MIN_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("SMS_ALL_PMIN2",F.col("SMS_ALL_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("SMS_ALL_PMIN3",F.col("SMS_ALL_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("SMS_ALL_PMIN4",F.col("SMS_ALL_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("SMS_TMHR_PMIN2",F.col("SMS_TMHR_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("SMS_TMHR_PMIN3",F.col("SMS_TMHR_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("SMS_TMHR_PMIN4",F.col("SMS_TMHR_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("SMS_BONBON_PMIN2",F.col("SMS_BONBON_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("SMS_BONBON_PMIN3",F.col("SMS_BONBON_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("SMS_BONBON_PMIN4",F.col("SMS_BONBON_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("SMS_VIP_PMIN2",F.col("SMS_VIP_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("SMS_VIP_PMIN3",F.col("SMS_VIP_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("SMS_VIP_PMIN4",F.col("SMS_VIP_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("SMS_TELE2_PMIN2",F.col("SMS_TELE2_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("SMS_TELE2_PMIN3",F.col("SMS_TELE2_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("SMS_TELE2_PMIN4",F.col("SMS_TELE2_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("SMS_TOMATO_PMIN2",F.col("SMS_TOMATO_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("SMS_TOMATO_PMIN3",F.col("SMS_TOMATO_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("SMS_TOMATO_PMIN4",F.col("SMS_TOMATO_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("SMS_INTERNATIONAL_PMIN2",F.col("SMS_INTERNATIONAL_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("SMS_INTERNATIONAL_PMIN3",F.col("SMS_INTERNATIONAL_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("SMS_INTERNATIONAL_PMIN4",F.col("SMS_INTERNATIONAL_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("SMS_VAS_PMIN2",F.col("SMS_VAS_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("SMS_VAS_PMIN3",F.col("SMS_VAS_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("SMS_VAS_PMIN4",F.col("SMS_VAS_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("SMS_ROAMING_PMIN2",F.col("SMS_ROAMING_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("SMS_ROAMING_PMIN3",F.col("SMS_ROAMING_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("SMS_ROAMING_PMIN4",F.col("SMS_ROAMING_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("DATA_NATIONAL_VOLUME_MB_PMIN2",F.col("DATA_NATIONAL_VOLUME_MB_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("DATA_NATIONAL_VOLUME_MB_PMIN3",F.col("DATA_NATIONAL_VOLUME_MB_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("DATA_NATIONAL_VOLUME_MB_PMIN4",F.col("DATA_NATIONAL_VOLUME_MB_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("DATA_NATIONAL_SESSIONS_PMIN2",F.col("DATA_NATIONAL_SESSIONS_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("DATA_NATIONAL_SESSIONS_PMIN3",F.col("DATA_NATIONAL_SESSIONS_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("DATA_NATIONAL_SESSIONS_PMIN4",F.col("DATA_NATIONAL_SESSIONS_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("DATA_ROAMING_VOLUME_MB_PMIN2",F.col("DATA_ROAMING_VOLUME_MB_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("DATA_ROAMING_VOLUME_MB_PMIN3",F.col("DATA_ROAMING_VOLUME_MB_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("DATA_ROAMING_VOLUME_MB_PMIN4",F.col("DATA_ROAMING_VOLUME_MB_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("DATA_ROAMING_SESSIONS_PMIN2",F.col("DATA_ROAMING_SESSIONS_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("DATA_ROAMING_SESSIONS_PMIN3",F.col("DATA_ROAMING_SESSIONS_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("DATA_ROAMING_SESSIONS_PMIN4",F.col("DATA_ROAMING_SESSIONS_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INT_CALL_IN_ADMIN_ACT_PMIN2",F.col("INT_CALL_IN_ADMIN_ACT_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INT_CALL_IN_ADMIN_ACT_PMIN3",F.col("INT_CALL_IN_ADMIN_ACT_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INT_CALL_IN_ADMIN_ACT_PMIN4",F.col("INT_CALL_IN_ADMIN_ACT_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INT_CALL_IN_HRC_PMIN2",F.col("INT_CALL_IN_HRC_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INT_CALL_IN_HRC_PMIN3",F.col("INT_CALL_IN_HRC_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INT_CALL_IN_HRC_PMIN4",F.col("INT_CALL_IN_HRC_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INT_CALL_IN_ICCA_TIM_PMIN2",F.col("INT_CALL_IN_ICCA_TIM_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INT_CALL_IN_ICCA_TIM_PMIN3",F.col("INT_CALL_IN_ICCA_TIM_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INT_CALL_IN_ICCA_TIM_PMIN4",F.col("INT_CALL_IN_ICCA_TIM_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INT_CALL_IN_INFO_PMIN2",F.col("INT_CALL_IN_INFO_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INT_CALL_IN_INFO_PMIN3",F.col("INT_CALL_IN_INFO_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INT_CALL_IN_INFO_PMIN4",F.col("INT_CALL_IN_INFO_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INT_CALL_IN_INTERACTION_PMIN2",F.col("INT_CALL_IN_INTERACTION_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INT_CALL_IN_INTERACTION_PMIN3",F.col("INT_CALL_IN_INTERACTION_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INT_CALL_IN_INTERACTION_PMIN4",F.col("INT_CALL_IN_INTERACTION_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INT_CALL_IN_USLUGE_KAMP_PMIN2",F.col("INT_CALL_IN_USLUGE_KAMP_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INT_CALL_IN_USLUGE_KAMP_PMIN3",F.col("INT_CALL_IN_USLUGE_KAMP_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INT_CALL_IN_USLUGE_KAMP_PMIN4",F.col("INT_CALL_IN_USLUGE_KAMP_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INT_CALL_IN_OSTALO_PMIN2",F.col("INT_CALL_IN_OSTALO_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INT_CALL_IN_OSTALO_PMIN3",F.col("INT_CALL_IN_OSTALO_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INT_CALL_IN_OSTALO_PMIN4",F.col("INT_CALL_IN_OSTALO_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INT_CALL_IN_RACUNI_PMIN2",F.col("INT_CALL_IN_RACUNI_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INT_CALL_IN_RACUNI_PMIN3",F.col("INT_CALL_IN_RACUNI_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INT_CALL_IN_RACUNI_PMIN4",F.col("INT_CALL_IN_RACUNI_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INT_CALL_IN_POS_KOR_PMIN2",F.col("INT_CALL_IN_POS_KOR_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INT_CALL_IN_POS_KOR_PMIN3",F.col("INT_CALL_IN_POS_KOR_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INT_CALL_IN_POS_KOR_PMIN4",F.col("INT_CALL_IN_POS_KOR_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INT_CALL_IN_PRIGOVOR_PMIN2",F.col("INT_CALL_IN_PRIGOVOR_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INT_CALL_IN_PRIGOVOR_PMIN3",F.col("INT_CALL_IN_PRIGOVOR_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INT_CALL_IN_PRIGOVOR_PMIN4",F.col("INT_CALL_IN_PRIGOVOR_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INT_CALL_IN_UGOVOR_PMIN2",F.col("INT_CALL_IN_UGOVOR_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INT_CALL_IN_UGOVOR_PMIN3",F.col("INT_CALL_IN_UGOVOR_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INT_CALL_IN_UGOVOR_PMIN4",F.col("INT_CALL_IN_UGOVOR_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INT_CALL_IN_SAVEDESK_PMIN2",F.col("INT_CALL_IN_SAVEDESK_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INT_CALL_IN_SAVEDESK_PMIN3",F.col("INT_CALL_IN_SAVEDESK_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INT_CALL_IN_SAVEDESK_PMIN4",F.col("INT_CALL_IN_SAVEDESK_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INT_CALL_IN_TEHNICKA_PMIN2",F.col("INT_CALL_IN_TEHNICKA_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INT_CALL_IN_TEHNICKA_PMIN3",F.col("INT_CALL_IN_TEHNICKA_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INT_CALL_IN_TEHNICKA_PMIN4",F.col("INT_CALL_IN_TEHNICKA_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INT_CALL_IN_TELESALES_PMIN2",F.col("INT_CALL_IN_TELESALES_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INT_CALL_IN_TELESALES_PMIN3",F.col("INT_CALL_IN_TELESALES_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INT_CALL_IN_TELESALES_PMIN4",F.col("INT_CALL_IN_TELESALES_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INT_CALL_IN_TRIM_TIM_PMIN2",F.col("INT_CALL_IN_TRIM_TIM_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INT_CALL_IN_TRIM_TIM_PMIN3",F.col("INT_CALL_IN_TRIM_TIM_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INT_CALL_IN_TRIM_TIM_PMIN4",F.col("INT_CALL_IN_TRIM_TIM_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INT_CALL_IN_X7_PMIN2",F.col("INT_CALL_IN_X7_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INT_CALL_IN_X7_PMIN3",F.col("INT_CALL_IN_X7_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INT_CALL_IN_X7_PMIN4",F.col("INT_CALL_IN_X7_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("QTY_DAYS_INTERACTING_PMIN2",F.col("QTY_DAYS_INTERACTING_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("QTY_DAYS_INTERACTING_PMIN3",F.col("QTY_DAYS_INTERACTING_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("QTY_DAYS_INTERACTING_PMIN4",F.col("QTY_DAYS_INTERACTING_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INVOICED_PMIN2",F.col("INVOICED_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INVOICED_PMIN3",F.col("INVOICED_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INVOICED_PMIN4",F.col("INVOICED_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("REVENUE_PMIN2",F.col("REVENUE_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("REVENUE_PMIN3",F.col("REVENUE_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("REVENUE_PMIN4",F.col("REVENUE_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("ROAMING_AMOUNT_PMIN2",F.col("ROAMING_AMOUNT_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("ROAMING_AMOUNT_PMIN3",F.col("ROAMING_AMOUNT_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("ROAMING_AMOUNT_PMIN4",F.col("ROAMING_AMOUNT_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("REVENUE_SUBSCRIPTION_PMIN2",F.col("REVENUE_SUBSCRIPTION_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("REVENUE_SUBSCRIPTION_PMIN3",F.col("REVENUE_SUBSCRIPTION_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("REVENUE_SUBSCRIPTION_PMIN4",F.col("REVENUE_SUBSCRIPTION_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("REVENUE_VOICE_PMIN2",F.col("REVENUE_VOICE_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("REVENUE_VOICE_PMIN3",F.col("REVENUE_VOICE_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("REVENUE_VOICE_PMIN4",F.col("REVENUE_VOICE_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("REVENUE_SMS_PMIN2",F.col("REVENUE_SMS_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("REVENUE_SMS_PMIN3",F.col("REVENUE_SMS_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("REVENUE_SMS_PMIN4",F.col("REVENUE_SMS_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("REVENUE_DATA_NATIONAL_PMIN2",F.col("REVENUE_DATA_NATIONAL_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("REVENUE_DATA_NATIONAL_PMIN3",F.col("REVENUE_DATA_NATIONAL_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("REVENUE_DATA_NATIONAL_PMIN4",F.col("REVENUE_DATA_NATIONAL_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("REVENUE_DATA_ROAMING_PMIN2",F.col("REVENUE_DATA_ROAMING_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("REVENUE_DATA_ROAMING_PMIN3",F.col("REVENUE_DATA_ROAMING_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("REVENUE_DATA_ROAMING_PMIN4",F.col("REVENUE_DATA_ROAMING_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("REVENUE_VOICE_VAS_PMIN2",F.col("REVENUE_VOICE_VAS_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("REVENUE_VOICE_VAS_PMIN3",F.col("REVENUE_VOICE_VAS_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("REVENUE_VOICE_VAS_PMIN4",F.col("REVENUE_VOICE_VAS_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("REVENUE_SMS_VAS_PMIN2",F.col("REVENUE_SMS_VAS_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("REVENUE_SMS_VAS_PMIN3",F.col("REVENUE_SMS_VAS_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("REVENUE_SMS_VAS_PMIN4",F.col("REVENUE_SMS_VAS_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("REVENUE_GSM_LATE_FEE_PMIN2",F.col("REVENUE_GSM_LATE_FEE_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("REVENUE_GSM_LATE_FEE_PMIN3",F.col("REVENUE_GSM_LATE_FEE_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("REVENUE_GSM_LATE_FEE_PMIN4",F.col("REVENUE_GSM_LATE_FEE_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("REVENUE_GSM_OTHER_PMIN2",F.col("REVENUE_GSM_OTHER_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("REVENUE_GSM_OTHER_PMIN3",F.col("REVENUE_GSM_OTHER_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("REVENUE_GSM_OTHER_PMIN4",F.col("REVENUE_GSM_OTHER_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_VOICE_ONNET_PMIN2",F.col("COST_VOICE_ONNET_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_VOICE_ONNET_PMIN3",F.col("COST_VOICE_ONNET_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_VOICE_ONNET_PMIN4",F.col("COST_VOICE_ONNET_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_VOICE_OFFNET_PMIN2",F.col("COST_VOICE_OFFNET_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_VOICE_OFFNET_PMIN3",F.col("COST_VOICE_OFFNET_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_VOICE_OFFNET_PMIN4",F.col("COST_VOICE_OFFNET_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_VOICE_VIP_PMIN2",F.col("COST_VOICE_VIP_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_VOICE_VIP_PMIN3",F.col("COST_VOICE_VIP_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_VOICE_VIP_PMIN4",F.col("COST_VOICE_VIP_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_VOICE_TELE2_PMIN2",F.col("COST_VOICE_TELE2_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_VOICE_TELE2_PMIN3",F.col("COST_VOICE_TELE2_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_VOICE_TELE2_PMIN4",F.col("COST_VOICE_TELE2_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_VOICE_TOMATO_PMIN2",F.col("COST_VOICE_TOMATO_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_VOICE_TOMATO_PMIN3",F.col("COST_VOICE_TOMATO_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_VOICE_TOMATO_PMIN4",F.col("COST_VOICE_TOMATO_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_VOICE_BONBON_PMIN2",F.col("COST_VOICE_BONBON_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_VOICE_BONBON_PMIN3",F.col("COST_VOICE_BONBON_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_VOICE_BONBON_PMIN4",F.col("COST_VOICE_BONBON_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_VOICE_FIX_PMIN2",F.col("COST_VOICE_FIX_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_VOICE_FIX_PMIN3",F.col("COST_VOICE_FIX_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_VOICE_FIX_PMIN4",F.col("COST_VOICE_FIX_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_VOICE_INT_PMIN2",F.col("COST_VOICE_INT_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_VOICE_INT_PMIN3",F.col("COST_VOICE_INT_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_VOICE_INT_PMIN4",F.col("COST_VOICE_INT_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_VOICE_ROAMING_PMIN2",F.col("COST_VOICE_ROAMING_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_VOICE_ROAMING_PMIN3",F.col("COST_VOICE_ROAMING_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_VOICE_ROAMING_PMIN4",F.col("COST_VOICE_ROAMING_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_VOICE_TOTAL_PMIN2",F.col("COST_VOICE_TOTAL_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_VOICE_TOTAL_PMIN3",F.col("COST_VOICE_TOTAL_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_VOICE_TOTAL_PMIN4",F.col("COST_VOICE_TOTAL_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_VOICE_VAS_PMIN2",F.col("COST_VOICE_VAS_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_VOICE_VAS_PMIN3",F.col("COST_VOICE_VAS_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_VOICE_VAS_PMIN4",F.col("COST_VOICE_VAS_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_SMS_ONNET_PMIN2",F.col("COST_SMS_ONNET_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_SMS_ONNET_PMIN3",F.col("COST_SMS_ONNET_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_SMS_ONNET_PMIN4",F.col("COST_SMS_ONNET_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_SMS_OFFNET_PMIN2",F.col("COST_SMS_OFFNET_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_SMS_OFFNET_PMIN3",F.col("COST_SMS_OFFNET_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_SMS_OFFNET_PMIN4",F.col("COST_SMS_OFFNET_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_SMS_VIP_PMIN2",F.col("COST_SMS_VIP_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_SMS_VIP_PMIN3",F.col("COST_SMS_VIP_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_SMS_VIP_PMIN4",F.col("COST_SMS_VIP_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_SMS_TELE2_PMIN2",F.col("COST_SMS_TELE2_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_SMS_TELE2_PMIN3",F.col("COST_SMS_TELE2_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_SMS_TELE2_PMIN4",F.col("COST_SMS_TELE2_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_SMS_TOMATO_PMIN2",F.col("COST_SMS_TOMATO_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_SMS_TOMATO_PMIN3",F.col("COST_SMS_TOMATO_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_SMS_TOMATO_PMIN4",F.col("COST_SMS_TOMATO_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_SMS_BONBON_PMIN2",F.col("COST_SMS_BONBON_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_SMS_BONBON_PMIN3",F.col("COST_SMS_BONBON_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_SMS_BONBON_PMIN4",F.col("COST_SMS_BONBON_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_SMS_TOTAL_PMIN2",F.col("COST_SMS_TOTAL_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_SMS_TOTAL_PMIN3",F.col("COST_SMS_TOTAL_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_SMS_TOTAL_PMIN4",F.col("COST_SMS_TOTAL_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_SMS_ROAMING_PMIN2",F.col("COST_SMS_ROAMING_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_SMS_ROAMING_PMIN3",F.col("COST_SMS_ROAMING_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_SMS_ROAMING_PMIN4",F.col("COST_SMS_ROAMING_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_MMS_TOTAL_PMIN2",F.col("COST_MMS_TOTAL_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_MMS_TOTAL_PMIN3",F.col("COST_MMS_TOTAL_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_MMS_TOTAL_PMIN4",F.col("COST_MMS_TOTAL_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_DATA_PMIN2",F.col("COST_DATA_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_DATA_PMIN3",F.col("COST_DATA_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_DATA_PMIN4",F.col("COST_DATA_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_DATA_NATIONAL_PMIN2",F.col("COST_DATA_NATIONAL_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_DATA_NATIONAL_PMIN3",F.col("COST_DATA_NATIONAL_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_DATA_NATIONAL_PMIN4",F.col("COST_DATA_NATIONAL_PMIN4").cast(IntegerType()))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [80]:
mob_09_tele_avg = mob_09_tele_avg.groupBy("oib","calendar_date").agg(

F.avg("VOICE_ALL_CALLS_PMIN2").alias("VOICE_ALL_CALLS_PMIN2"),
F.avg("VOICE_ALL_CALLS_PMIN3").alias("VOICE_ALL_CALLS_PMIN3"),
F.avg("VOICE_ALL_CALLS_PMIN4").alias("VOICE_ALL_CALLS_PMIN4"),
F.avg("VOICE_TMHR_CALLS_PMIN2").alias("VOICE_TMHR_CALLS_PMIN2"),
F.avg("VOICE_TMHR_CALLS_PMIN3").alias("VOICE_TMHR_CALLS_PMIN3"),
F.avg("VOICE_TMHR_CALLS_PMIN4").alias("VOICE_TMHR_CALLS_PMIN4"),
F.avg("VOICE_BONBON_CALLS_PMIN2").alias("VOICE_BONBON_CALLS_PMIN2"),
F.avg("VOICE_BONBON_CALLS_PMIN3").alias("VOICE_BONBON_CALLS_PMIN3"),
F.avg("VOICE_BONBON_CALLS_PMIN4").alias("VOICE_BONBON_CALLS_PMIN4"),
F.avg("VOICE_VIP_CALLS_PMIN2").alias("VOICE_VIP_CALLS_PMIN2"),
F.avg("VOICE_VIP_CALLS_PMIN3").alias("VOICE_VIP_CALLS_PMIN3"),
F.avg("VOICE_VIP_CALLS_PMIN4").alias("VOICE_VIP_CALLS_PMIN4"),
F.avg("VOICE_TELE2_CALLS_PMIN2").alias("VOICE_TELE2_CALLS_PMIN2"),
F.avg("VOICE_TELE2_CALLS_PMIN3").alias("VOICE_TELE2_CALLS_PMIN3"),
F.avg("VOICE_TELE2_CALLS_PMIN4").alias("VOICE_TELE2_CALLS_PMIN4"),
F.avg("VOICE_TOMATO_CALLS_PMIN2").alias("VOICE_TOMATO_CALLS_PMIN2"),
F.avg("VOICE_TOMATO_CALLS_PMIN3").alias("VOICE_TOMATO_CALLS_PMIN3"),
F.avg("VOICE_TOMATO_CALLS_PMIN4").alias("VOICE_TOMATO_CALLS_PMIN4"),
F.avg("VOICE_INTRNTIONAL_CALLS_PMIN2").alias("VOICE_INTRNTIONAL_CALLS_PMIN2"),
F.avg("VOICE_INTRNTIONAL_CALLS_PMIN3").alias("VOICE_INTRNTIONAL_CALLS_PMIN3"),
F.avg("VOICE_INTRNTIONAL_CALLS_PMIN4").alias("VOICE_INTRNTIONAL_CALLS_PMIN4"),
F.avg("VOICE_FIXED_CALLS_MIN2").alias("VOICE_FIXED_CALLS_MIN2"),
F.avg("VOICE_FIXED_CALLS_MIN3").alias("VOICE_FIXED_CALLS_MIN3"),
F.avg("VOICE_FIXED_CALLS_MIN4").alias("VOICE_FIXED_CALLS_MIN4"),
F.avg("VOICE_VAS_CALLS_PMIN2").alias("VOICE_VAS_CALLS_PMIN2"),
F.avg("VOICE_VAS_CALLS_PMIN3").alias("VOICE_VAS_CALLS_PMIN3"),
F.avg("VOICE_VAS_CALLS_PMIN4").alias("VOICE_VAS_CALLS_PMIN4"),
F.avg("VOICE_ROAMING_CALLS_PMIN2").alias("VOICE_ROAMING_CALLS_PMIN2"),
F.avg("VOICE_ROAMING_CALLS_PMIN3").alias("VOICE_ROAMING_CALLS_PMIN3"),
F.avg("VOICE_ROAMING_CALLS_PMIN4").alias("VOICE_ROAMING_CALLS_PMIN4"),
F.avg("VOICE_ALL_MIN_PMIN2").alias("VOICE_ALL_MIN_PMIN2"),
F.avg("VOICE_ALL_MIN_PMIN3").alias("VOICE_ALL_MIN_PMIN3"),
F.avg("VOICE_ALL_MIN_PMIN4").alias("VOICE_ALL_MIN_PMIN4"),
F.avg("VOICE_TMHR_MIN_PMIN2").alias("VOICE_TMHR_MIN_PMIN2"),
F.avg("VOICE_TMHR_MIN_PMIN3").alias("VOICE_TMHR_MIN_PMIN3"),
F.avg("VOICE_TMHR_MIN_PMIN4").alias("VOICE_TMHR_MIN_PMIN4"),
# F.avg("VOICE_BONBON_MIN_PMIN_2").alias("VOICE_BONBON_MIN_PMIN_2"),
# F.avg("VOICE_BONBON_MIN_PMIN_3").alias("VOICE_BONBON_MIN_PMIN_3"),
# F.avg("VOICE_BONBON_MIN_PMIN_4").alias("VOICE_BONBON_MIN_PMIN_4"),
F.avg("VOICE_VIP_MIN_PMIN2").alias("VOICE_VIP_MIN_PMIN2"),
F.avg("VOICE_VIP_MIN_PMIN3").alias("VOICE_VIP_MIN_PMIN3"),
F.avg("VOICE_VIP_MIN_PMIN4").alias("VOICE_VIP_MIN_PMIN4"),
F.avg("VOICE_TELE2_MIN_PMIN2").alias("VOICE_TELE2_MIN_PMIN2"),
F.avg("VOICE_TELE2_MIN_PMIN3").alias("VOICE_TELE2_MIN_PMIN3"),
F.avg("VOICE_TELE2_MIN_PMIN4").alias("VOICE_TELE2_MIN_PMIN4"),
F.avg("VOICE_TOMATO_MIN_PMIN2").alias("VOICE_TOMATO_MIN_PMIN2"),
F.avg("VOICE_TOMATO_MIN_PMIN3").alias("VOICE_TOMATO_MIN_PMIN3"),
F.avg("VOICE_TOMATO_MIN_PMIN4").alias("VOICE_TOMATO_MIN_PMIN4"),
F.avg("VOICE_INTERNATIONAL_MIN_PMIN2").alias("VOICE_INTERNATIONAL_MIN_PMIN2"),
F.avg("VOICE_INTERNATIONAL_MIN_PMIN3").alias("VOICE_INTERNATIONAL_MIN_PMIN3"),
F.avg("VOICE_INTERNATIONAL_MIN_PMIN4").alias("VOICE_INTERNATIONAL_MIN_PMIN4"),
F.avg("VOICE_FIXED_MIN_PMIN2").alias("VOICE_FIXED_MIN_PMIN2"),
F.avg("VOICE_FIXED_MIN_PMIN3").alias("VOICE_FIXED_MIN_PMIN3"),
F.avg("VOICE_FIXED_MIN_PMIN4").alias("VOICE_FIXED_MIN_PMIN4"),
F.avg("VOICE_VAS_MIN_PMIN2").alias("VOICE_VAS_MIN_PMIN2"),
F.avg("VOICE_VAS_MIN_PMIN3").alias("VOICE_VAS_MIN_PMIN3"),
F.avg("VOICE_VAS_MIN_PMIN4").alias("VOICE_VAS_MIN_PMIN4"),
F.avg("VOICE_ROAMING_MIN_PMIN2").alias("VOICE_ROAMING_MIN_PMIN2"),
F.avg("VOICE_ROAMING_MIN_PMIN3").alias("VOICE_ROAMING_MIN_PMIN3"),
F.avg("VOICE_ROAMING_MIN_PMIN4").alias("VOICE_ROAMING_MIN_PMIN4"),
F.avg("SMS_ALL_PMIN2").alias("SMS_ALL_PMIN2"),
F.avg("SMS_ALL_PMIN3").alias("SMS_ALL_PMIN3"),
F.avg("SMS_ALL_PMIN4").alias("SMS_ALL_PMIN4"),
F.avg("SMS_TMHR_PMIN2").alias("SMS_TMHR_PMIN2"),
F.avg("SMS_TMHR_PMIN3").alias("SMS_TMHR_PMIN3"),
F.avg("SMS_TMHR_PMIN4").alias("SMS_TMHR_PMIN4"),
F.avg("SMS_BONBON_PMIN2").alias("SMS_BONBON_PMIN2"),
F.avg("SMS_BONBON_PMIN3").alias("SMS_BONBON_PMIN3"),
F.avg("SMS_BONBON_PMIN4").alias("SMS_BONBON_PMIN4"),
F.avg("SMS_VIP_PMIN2").alias("SMS_VIP_PMIN2"),
F.avg("SMS_VIP_PMIN3").alias("SMS_VIP_PMIN3"),
F.avg("SMS_VIP_PMIN4").alias("SMS_VIP_PMIN4"),
F.avg("SMS_TELE2_PMIN2").alias("SMS_TELE2_PMIN2"),
F.avg("SMS_TELE2_PMIN3").alias("SMS_TELE2_PMIN3"),
F.avg("SMS_TELE2_PMIN4").alias("SMS_TELE2_PMIN4"),
F.avg("SMS_TOMATO_PMIN2").alias("SMS_TOMATO_PMIN2"),
F.avg("SMS_TOMATO_PMIN3").alias("SMS_TOMATO_PMIN3"),
F.avg("SMS_TOMATO_PMIN4").alias("SMS_TOMATO_PMIN4"),
F.avg("SMS_INTERNATIONAL_PMIN2").alias("SMS_INTERNATIONAL_PMIN2"),
F.avg("SMS_INTERNATIONAL_PMIN3").alias("SMS_INTERNATIONAL_PMIN3"),
F.avg("SMS_INTERNATIONAL_PMIN4").alias("SMS_INTERNATIONAL_PMIN4"),
F.avg("SMS_VAS_PMIN2").alias("SMS_VAS_PMIN2"),
F.avg("SMS_VAS_PMIN3").alias("SMS_VAS_PMIN3"),
F.avg("SMS_VAS_PMIN4").alias("SMS_VAS_PMIN4"),
F.avg("SMS_ROAMING_PMIN2").alias("SMS_ROAMING_PMIN2"),
F.avg("SMS_ROAMING_PMIN3").alias("SMS_ROAMING_PMIN3"),
F.avg("SMS_ROAMING_PMIN4").alias("SMS_ROAMING_PMIN4"),
F.avg("DATA_NATIONAL_VOLUME_MB_PMIN2").alias("DATA_NATIONAL_VOLUME_MB_PMIN2"),
F.avg("DATA_NATIONAL_VOLUME_MB_PMIN3").alias("DATA_NATIONAL_VOLUME_MB_PMIN3"),
F.avg("DATA_NATIONAL_VOLUME_MB_PMIN4").alias("DATA_NATIONAL_VOLUME_MB_PMIN4"),
F.avg("DATA_NATIONAL_SESSIONS_PMIN2").alias("DATA_NATIONAL_SESSIONS_PMIN2"),
F.avg("DATA_NATIONAL_SESSIONS_PMIN3").alias("DATA_NATIONAL_SESSIONS_PMIN3"),
F.avg("DATA_NATIONAL_SESSIONS_PMIN4").alias("DATA_NATIONAL_SESSIONS_PMIN4"),
F.avg("DATA_ROAMING_VOLUME_MB_PMIN2").alias("DATA_ROAMING_VOLUME_MB_PMIN2"),
F.avg("DATA_ROAMING_VOLUME_MB_PMIN3").alias("DATA_ROAMING_VOLUME_MB_PMIN3"),
F.avg("DATA_ROAMING_VOLUME_MB_PMIN4").alias("DATA_ROAMING_VOLUME_MB_PMIN4"),
F.avg("DATA_ROAMING_SESSIONS_PMIN2").alias("DATA_ROAMING_SESSIONS_PMIN2"),
F.avg("DATA_ROAMING_SESSIONS_PMIN3").alias("DATA_ROAMING_SESSIONS_PMIN3"),
F.avg("DATA_ROAMING_SESSIONS_PMIN4").alias("DATA_ROAMING_SESSIONS_PMIN4"),
F.avg("INT_CALL_IN_ADMIN_ACT_PMIN2").alias("INT_CALL_IN_ADMIN_ACT_PMIN2"),
F.avg("INT_CALL_IN_ADMIN_ACT_PMIN3").alias("INT_CALL_IN_ADMIN_ACT_PMIN3"),
F.avg("INT_CALL_IN_ADMIN_ACT_PMIN4").alias("INT_CALL_IN_ADMIN_ACT_PMIN4"),
F.avg("INT_CALL_IN_HRC_PMIN2").alias("INT_CALL_IN_HRC_PMIN2"),
F.avg("INT_CALL_IN_HRC_PMIN3").alias("INT_CALL_IN_HRC_PMIN3"),
F.avg("INT_CALL_IN_HRC_PMIN4").alias("INT_CALL_IN_HRC_PMIN4"),
F.avg("INT_CALL_IN_ICCA_TIM_PMIN2").alias("INT_CALL_IN_ICCA_TIM_PMIN2"),
F.avg("INT_CALL_IN_ICCA_TIM_PMIN3").alias("INT_CALL_IN_ICCA_TIM_PMIN3"),
F.avg("INT_CALL_IN_ICCA_TIM_PMIN4").alias("INT_CALL_IN_ICCA_TIM_PMIN4"),
F.avg("INT_CALL_IN_INFO_PMIN2").alias("INT_CALL_IN_INFO_PMIN2"),
F.avg("INT_CALL_IN_INFO_PMIN3").alias("INT_CALL_IN_INFO_PMIN3"),
F.avg("INT_CALL_IN_INFO_PMIN4").alias("INT_CALL_IN_INFO_PMIN4"),
F.avg("INT_CALL_IN_INTERACTION_PMIN2").alias("INT_CALL_IN_INTERACTION_PMIN2"),
F.avg("INT_CALL_IN_INTERACTION_PMIN3").alias("INT_CALL_IN_INTERACTION_PMIN3"),
F.avg("INT_CALL_IN_INTERACTION_PMIN4").alias("INT_CALL_IN_INTERACTION_PMIN4"),
F.avg("INT_CALL_IN_USLUGE_KAMP_PMIN2").alias("INT_CALL_IN_USLUGE_KAMP_PMIN2"),
F.avg("INT_CALL_IN_USLUGE_KAMP_PMIN3").alias("INT_CALL_IN_USLUGE_KAMP_PMIN3"),
F.avg("INT_CALL_IN_USLUGE_KAMP_PMIN4").alias("INT_CALL_IN_USLUGE_KAMP_PMIN4"),
F.avg("INT_CALL_IN_OSTALO_PMIN2").alias("INT_CALL_IN_OSTALO_PMIN2"),
F.avg("INT_CALL_IN_OSTALO_PMIN3").alias("INT_CALL_IN_OSTALO_PMIN3"),
F.avg("INT_CALL_IN_OSTALO_PMIN4").alias("INT_CALL_IN_OSTALO_PMIN4"),
F.avg("INT_CALL_IN_RACUNI_PMIN2").alias("INT_CALL_IN_RACUNI_PMIN2"),
F.avg("INT_CALL_IN_RACUNI_PMIN3").alias("INT_CALL_IN_RACUNI_PMIN3"),
F.avg("INT_CALL_IN_RACUNI_PMIN4").alias("INT_CALL_IN_RACUNI_PMIN4"),
F.avg("INT_CALL_IN_POS_KOR_PMIN2").alias("INT_CALL_IN_POS_KOR_PMIN2"),
F.avg("INT_CALL_IN_POS_KOR_PMIN3").alias("INT_CALL_IN_POS_KOR_PMIN3"),
F.avg("INT_CALL_IN_POS_KOR_PMIN4").alias("INT_CALL_IN_POS_KOR_PMIN4"),
F.avg("INT_CALL_IN_PRIGOVOR_PMIN2").alias("INT_CALL_IN_PRIGOVOR_PMIN2"),
F.avg("INT_CALL_IN_PRIGOVOR_PMIN3").alias("INT_CALL_IN_PRIGOVOR_PMIN3"),
F.avg("INT_CALL_IN_PRIGOVOR_PMIN4").alias("INT_CALL_IN_PRIGOVOR_PMIN4"),
F.avg("INT_CALL_IN_UGOVOR_PMIN2").alias("INT_CALL_IN_UGOVOR_PMIN2"),
F.avg("INT_CALL_IN_UGOVOR_PMIN3").alias("INT_CALL_IN_UGOVOR_PMIN3"),
F.avg("INT_CALL_IN_UGOVOR_PMIN4").alias("INT_CALL_IN_UGOVOR_PMIN4"),
F.avg("INT_CALL_IN_SAVEDESK_PMIN2").alias("INT_CALL_IN_SAVEDESK_PMIN2"),
F.avg("INT_CALL_IN_SAVEDESK_PMIN3").alias("INT_CALL_IN_SAVEDESK_PMIN3"),
F.avg("INT_CALL_IN_SAVEDESK_PMIN4").alias("INT_CALL_IN_SAVEDESK_PMIN4"),
F.avg("INT_CALL_IN_TEHNICKA_PMIN2").alias("INT_CALL_IN_TEHNICKA_PMIN2"),
F.avg("INT_CALL_IN_TEHNICKA_PMIN3").alias("INT_CALL_IN_TEHNICKA_PMIN3"),
F.avg("INT_CALL_IN_TEHNICKA_PMIN4").alias("INT_CALL_IN_TEHNICKA_PMIN4"),
F.avg("INT_CALL_IN_TELESALES_PMIN2").alias("INT_CALL_IN_TELESALES_PMIN2"),
F.avg("INT_CALL_IN_TELESALES_PMIN3").alias("INT_CALL_IN_TELESALES_PMIN3"),
F.avg("INT_CALL_IN_TELESALES_PMIN4").alias("INT_CALL_IN_TELESALES_PMIN4"),
F.avg("INT_CALL_IN_TRIM_TIM_PMIN2").alias("INT_CALL_IN_TRIM_TIM_PMIN2"),
F.avg("INT_CALL_IN_TRIM_TIM_PMIN3").alias("INT_CALL_IN_TRIM_TIM_PMIN3"),
F.avg("INT_CALL_IN_TRIM_TIM_PMIN4").alias("INT_CALL_IN_TRIM_TIM_PMIN4"),
F.avg("INT_CALL_IN_X7_PMIN2").alias("INT_CALL_IN_X7_PMIN2"),
F.avg("INT_CALL_IN_X7_PMIN3").alias("INT_CALL_IN_X7_PMIN3"),
F.avg("INT_CALL_IN_X7_PMIN4").alias("INT_CALL_IN_X7_PMIN4"),
F.avg("QTY_DAYS_INTERACTING_PMIN2").alias("QTY_DAYS_INTERACTING_PMIN2"),
F.avg("QTY_DAYS_INTERACTING_PMIN3").alias("QTY_DAYS_INTERACTING_PMIN3"),
F.avg("QTY_DAYS_INTERACTING_PMIN4").alias("QTY_DAYS_INTERACTING_PMIN4"),
F.avg("INVOICED_PMIN2").alias("INVOICED_PMIN2"),
F.avg("INVOICED_PMIN3").alias("INVOICED_PMIN3"),
F.avg("INVOICED_PMIN4").alias("INVOICED_PMIN4"),
F.avg("REVENUE_PMIN2").alias("REVENUE_PMIN2"),
F.avg("REVENUE_PMIN3").alias("REVENUE_PMIN3"),
F.avg("REVENUE_PMIN4").alias("REVENUE_PMIN4"),
F.avg("ROAMING_AMOUNT_PMIN2").alias("ROAMING_AMOUNT_PMIN2"),
F.avg("ROAMING_AMOUNT_PMIN3").alias("ROAMING_AMOUNT_PMIN3"),
F.avg("ROAMING_AMOUNT_PMIN4").alias("ROAMING_AMOUNT_PMIN4"),
F.avg("REVENUE_SUBSCRIPTION_PMIN2").alias("REVENUE_SUBSCRIPTION_PMIN2"),
F.avg("REVENUE_SUBSCRIPTION_PMIN3").alias("REVENUE_SUBSCRIPTION_PMIN3"),
F.avg("REVENUE_SUBSCRIPTION_PMIN4").alias("REVENUE_SUBSCRIPTION_PMIN4"),
F.avg("REVENUE_VOICE_PMIN2").alias("REVENUE_VOICE_PMIN2"),
F.avg("REVENUE_VOICE_PMIN3").alias("REVENUE_VOICE_PMIN3"),
F.avg("REVENUE_VOICE_PMIN4").alias("REVENUE_VOICE_PMIN4"),
F.avg("REVENUE_SMS_PMIN2").alias("REVENUE_SMS_PMIN2"),
F.avg("REVENUE_SMS_PMIN3").alias("REVENUE_SMS_PMIN3"),
F.avg("REVENUE_SMS_PMIN4").alias("REVENUE_SMS_PMIN4"),
F.avg("REVENUE_DATA_NATIONAL_PMIN2").alias("REVENUE_DATA_NATIONAL_PMIN2"),
F.avg("REVENUE_DATA_NATIONAL_PMIN3").alias("REVENUE_DATA_NATIONAL_PMIN3"),
F.avg("REVENUE_DATA_NATIONAL_PMIN4").alias("REVENUE_DATA_NATIONAL_PMIN4"),
F.avg("REVENUE_DATA_ROAMING_PMIN2").alias("REVENUE_DATA_ROAMING_PMIN2"),
F.avg("REVENUE_DATA_ROAMING_PMIN3").alias("REVENUE_DATA_ROAMING_PMIN3"),
F.avg("REVENUE_DATA_ROAMING_PMIN4").alias("REVENUE_DATA_ROAMING_PMIN4"),
F.avg("REVENUE_VOICE_VAS_PMIN2").alias("REVENUE_VOICE_VAS_PMIN2"),
F.avg("REVENUE_VOICE_VAS_PMIN3").alias("REVENUE_VOICE_VAS_PMIN3"),
F.avg("REVENUE_VOICE_VAS_PMIN4").alias("REVENUE_VOICE_VAS_PMIN4"),
F.avg("REVENUE_SMS_VAS_PMIN2").alias("REVENUE_SMS_VAS_PMIN2"),
F.avg("REVENUE_SMS_VAS_PMIN3").alias("REVENUE_SMS_VAS_PMIN3"),
F.avg("REVENUE_SMS_VAS_PMIN4").alias("REVENUE_SMS_VAS_PMIN4"),
F.avg("REVENUE_GSM_LATE_FEE_PMIN2").alias("REVENUE_GSM_LATE_FEE_PMIN2"),
F.avg("REVENUE_GSM_LATE_FEE_PMIN3").alias("REVENUE_GSM_LATE_FEE_PMIN3"),
F.avg("REVENUE_GSM_LATE_FEE_PMIN4").alias("REVENUE_GSM_LATE_FEE_PMIN4"),
F.avg("REVENUE_GSM_OTHER_PMIN2").alias("REVENUE_GSM_OTHER_PMIN2"),
F.avg("REVENUE_GSM_OTHER_PMIN3").alias("REVENUE_GSM_OTHER_PMIN3"),
F.avg("REVENUE_GSM_OTHER_PMIN4").alias("REVENUE_GSM_OTHER_PMIN4"),
F.avg("COST_VOICE_ONNET_PMIN2").alias("COST_VOICE_ONNET_PMIN2"),
F.avg("COST_VOICE_ONNET_PMIN3").alias("COST_VOICE_ONNET_PMIN3"),
F.avg("COST_VOICE_ONNET_PMIN4").alias("COST_VOICE_ONNET_PMIN4"),
F.avg("COST_VOICE_OFFNET_PMIN2").alias("COST_VOICE_OFFNET_PMIN2"),
F.avg("COST_VOICE_OFFNET_PMIN3").alias("COST_VOICE_OFFNET_PMIN3"),
F.avg("COST_VOICE_OFFNET_PMIN4").alias("COST_VOICE_OFFNET_PMIN4"),
F.avg("COST_VOICE_VIP_PMIN2").alias("COST_VOICE_VIP_PMIN2"),
F.avg("COST_VOICE_VIP_PMIN3").alias("COST_VOICE_VIP_PMIN3"),
F.avg("COST_VOICE_VIP_PMIN4").alias("COST_VOICE_VIP_PMIN4"),
F.avg("COST_VOICE_TELE2_PMIN2").alias("COST_VOICE_TELE2_PMIN2"),
F.avg("COST_VOICE_TELE2_PMIN3").alias("COST_VOICE_TELE2_PMIN3"),
F.avg("COST_VOICE_TELE2_PMIN4").alias("COST_VOICE_TELE2_PMIN4"),
F.avg("COST_VOICE_TOMATO_PMIN2").alias("COST_VOICE_TOMATO_PMIN2"),
F.avg("COST_VOICE_TOMATO_PMIN3").alias("COST_VOICE_TOMATO_PMIN3"),
F.avg("COST_VOICE_TOMATO_PMIN4").alias("COST_VOICE_TOMATO_PMIN4"),
F.avg("COST_VOICE_BONBON_PMIN2").alias("COST_VOICE_BONBON_PMIN2"),
F.avg("COST_VOICE_BONBON_PMIN3").alias("COST_VOICE_BONBON_PMIN3"),
F.avg("COST_VOICE_BONBON_PMIN4").alias("COST_VOICE_BONBON_PMIN4"),
F.avg("COST_VOICE_FIX_PMIN2").alias("COST_VOICE_FIX_PMIN2"),
F.avg("COST_VOICE_FIX_PMIN3").alias("COST_VOICE_FIX_PMIN3"),
F.avg("COST_VOICE_FIX_PMIN4").alias("COST_VOICE_FIX_PMIN4"),
F.avg("COST_VOICE_INT_PMIN2").alias("COST_VOICE_INT_PMIN2"),
F.avg("COST_VOICE_INT_PMIN3").alias("COST_VOICE_INT_PMIN3"),
F.avg("COST_VOICE_INT_PMIN4").alias("COST_VOICE_INT_PMIN4"),
F.avg("COST_VOICE_ROAMING_PMIN2").alias("COST_VOICE_ROAMING_PMIN2"),
F.avg("COST_VOICE_ROAMING_PMIN3").alias("COST_VOICE_ROAMING_PMIN3"),
F.avg("COST_VOICE_ROAMING_PMIN4").alias("COST_VOICE_ROAMING_PMIN4"),
F.avg("COST_VOICE_TOTAL_PMIN2").alias("COST_VOICE_TOTAL_PMIN2"),
F.avg("COST_VOICE_TOTAL_PMIN3").alias("COST_VOICE_TOTAL_PMIN3"),
F.avg("COST_VOICE_TOTAL_PMIN4").alias("COST_VOICE_TOTAL_PMIN4"),
F.avg("COST_VOICE_VAS_PMIN2").alias("COST_VOICE_VAS_PMIN2"),
F.avg("COST_VOICE_VAS_PMIN3").alias("COST_VOICE_VAS_PMIN3"),
F.avg("COST_VOICE_VAS_PMIN4").alias("COST_VOICE_VAS_PMIN4"),
F.avg("COST_SMS_ONNET_PMIN2").alias("COST_SMS_ONNET_PMIN2"),
F.avg("COST_SMS_ONNET_PMIN3").alias("COST_SMS_ONNET_PMIN3"),
F.avg("COST_SMS_ONNET_PMIN4").alias("COST_SMS_ONNET_PMIN4"),
F.avg("COST_SMS_OFFNET_PMIN2").alias("COST_SMS_OFFNET_PMIN2"),
F.avg("COST_SMS_OFFNET_PMIN3").alias("COST_SMS_OFFNET_PMIN3"),
F.avg("COST_SMS_OFFNET_PMIN4").alias("COST_SMS_OFFNET_PMIN4"),
F.avg("COST_SMS_VIP_PMIN2").alias("COST_SMS_VIP_PMIN2"),
F.avg("COST_SMS_VIP_PMIN3").alias("COST_SMS_VIP_PMIN3"),
F.avg("COST_SMS_VIP_PMIN4").alias("COST_SMS_VIP_PMIN4"),
F.avg("COST_SMS_TELE2_PMIN2").alias("COST_SMS_TELE2_PMIN2"),
F.avg("COST_SMS_TELE2_PMIN3").alias("COST_SMS_TELE2_PMIN3"),
F.avg("COST_SMS_TELE2_PMIN4").alias("COST_SMS_TELE2_PMIN4"),
F.avg("COST_SMS_TOMATO_PMIN2").alias("COST_SMS_TOMATO_PMIN2"),
F.avg("COST_SMS_TOMATO_PMIN3").alias("COST_SMS_TOMATO_PMIN3"),
F.avg("COST_SMS_TOMATO_PMIN4").alias("COST_SMS_TOMATO_PMIN4"),
F.avg("COST_SMS_BONBON_PMIN2").alias("COST_SMS_BONBON_PMIN2"),
F.avg("COST_SMS_BONBON_PMIN3").alias("COST_SMS_BONBON_PMIN3"),
F.avg("COST_SMS_BONBON_PMIN4").alias("COST_SMS_BONBON_PMIN4"),
F.avg("COST_SMS_TOTAL_PMIN2").alias("COST_SMS_TOTAL_PMIN2"),
F.avg("COST_SMS_TOTAL_PMIN3").alias("COST_SMS_TOTAL_PMIN3"),
F.avg("COST_SMS_TOTAL_PMIN4").alias("COST_SMS_TOTAL_PMIN4"),
F.avg("COST_SMS_ROAMING_PMIN2").alias("COST_SMS_ROAMING_PMIN2"),
F.avg("COST_SMS_ROAMING_PMIN3").alias("COST_SMS_ROAMING_PMIN3"),
F.avg("COST_SMS_ROAMING_PMIN4").alias("COST_SMS_ROAMING_PMIN4"),
F.avg("COST_MMS_TOTAL_PMIN2").alias("COST_MMS_TOTAL_PMIN2"),
F.avg("COST_MMS_TOTAL_PMIN3").alias("COST_MMS_TOTAL_PMIN3"),
F.avg("COST_MMS_TOTAL_PMIN4").alias("COST_MMS_TOTAL_PMIN4"),
F.avg("COST_DATA_PMIN2").alias("COST_DATA_PMIN2"),
F.avg("COST_DATA_PMIN3").alias("COST_DATA_PMIN3"),
F.avg("COST_DATA_PMIN4").alias("COST_DATA_PMIN4"),
F.avg("COST_DATA_NATIONAL_PMIN2").alias("COST_DATA_NATIONAL_PMIN2"),
F.avg("COST_DATA_NATIONAL_PMIN3").alias("COST_DATA_NATIONAL_PMIN3"),
F.avg("COST_DATA_NATIONAL_PMIN4").alias("COST_DATA_NATIONAL_PMIN4"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [81]:
mob_09_tele_max.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

7318347

In [82]:
L4 = [
'GENDER',
'AGE',
#'RATE_PLAN_ID_PMIN2',
'HH_TYPE_PMIN2',
'HH_TYPE_PMIN3',
'HH_TYPE_PMIN4',
'M1_STATUS_PMIN2',
'M1_STATUS_PMIN3',
'M1_STATUS_PMIN4'
]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [84]:
mob_09_tele_first = mob_09_tele.select("oib","calendar_date",*L4)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [85]:
mob_09_tele_first = mob_09_tele_first.groupBy("oib","calendar_date").agg(
F.first("GENDER").alias("GENDER"),
F.first("AGE").alias("AGE"),
#F.first("RATE_PLAN_ID_PMIN2").alias("RATE_PLAN_ID_PMIN2"),
F.first("HH_TYPE_PMIN2").alias("HH_TYPE_PMIN2"),
F.first("HH_TYPE_PMIN3").alias("HH_TYPE_PMIN3"),
F.first("HH_TYPE_PMIN4").alias("HH_TYPE_PMIN4"),
F.first("M1_STATUS_PMIN2").alias("M1_STATUS_PMIN2"),
F.first("M1_STATUS_PMIN3").alias("M1_STATUS_PMIN3"),
F.first("M1_STATUS_PMIN4").alias("M1_STATUS_PMIN4"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [86]:
L5 = [
    'DATA_ONLY_TARIFF',
'HAS_FIXED_VOICE_HH_PMIN2',
'HAS_FIXED_VOICE_HH_PMIN3',
'HAS_FIXED_VOICE_HH_PMIN4',
'HAS_FIXED_BB_HH_PMIN2',
'HAS_FIXED_BB_HH_PMIN3',
'HAS_FIXED_BB_HH_PMIN4',
'HAS_FIXED_TV_HH_PMIN2',
'HAS_FIXED_TV_HH_PMIN3',
'HAS_FIXED_TV_HH_PMIN4',
'FULL_TO_PARTIAL_MAGENTA_PMIN43',
'FULL_TO_PARTIAL_MAGENTA_PMIN32',
'FULL_TO_NO_MAGENTA_PMIN43',
'FULL_TO_NO_MAGENTA_PMIN32',
'PARTIAL_TO_NO_MAGENTA_PMIN43',
'PARTIAL_TO_NO_MAGENTA_PMIN32'
    
]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [87]:
mob_09_tele_yes = mob_09_tele.select("oib","calendar_date",*L5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [88]:
mob_09_tele_yes

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[oib: string, calendar_date: date, DATA_ONLY_TARIFF: string, HAS_FIXED_VOICE_HH_PMIN2: string, HAS_FIXED_VOICE_HH_PMIN3: string, HAS_FIXED_VOICE_HH_PMIN4: string, HAS_FIXED_BB_HH_PMIN2: string, HAS_FIXED_BB_HH_PMIN3: string, HAS_FIXED_BB_HH_PMIN4: string, HAS_FIXED_TV_HH_PMIN2: string, HAS_FIXED_TV_HH_PMIN3: string, HAS_FIXED_TV_HH_PMIN4: string, FULL_TO_PARTIAL_MAGENTA_PMIN43: string, FULL_TO_PARTIAL_MAGENTA_PMIN32: string, FULL_TO_NO_MAGENTA_PMIN43: string, FULL_TO_NO_MAGENTA_PMIN32: string, PARTIAL_TO_NO_MAGENTA_PMIN43: string, PARTIAL_TO_NO_MAGENTA_PMIN32: string]

In [89]:
mob_09_tele_yes = mob_09_tele_yes.withColumn("DATA_ONLY_TARIFF",F.when((mob_09_tele_yes.DATA_ONLY_TARIFF == "Y") | (mob_09_tele_yes.DATA_ONLY_TARIFF == "1"),1).otherwise(0))
mob_09_tele_yes = mob_09_tele_yes.withColumn("HAS_FIXED_VOICE_HH_PMIN2",F.when((mob_09_tele_yes.HAS_FIXED_VOICE_HH_PMIN2 == "Y") | (mob_09_tele_yes.HAS_FIXED_VOICE_HH_PMIN2 == "1"),1).otherwise(0))
mob_09_tele_yes = mob_09_tele_yes.withColumn("HAS_FIXED_VOICE_HH_PMIN3",F.when((mob_09_tele_yes.HAS_FIXED_VOICE_HH_PMIN3 == "Y") | (mob_09_tele_yes.HAS_FIXED_VOICE_HH_PMIN3 == "1"),1).otherwise(0))
mob_09_tele_yes = mob_09_tele_yes.withColumn("HAS_FIXED_VOICE_HH_PMIN4",F.when((mob_09_tele_yes.HAS_FIXED_VOICE_HH_PMIN4 == "Y") | (mob_09_tele_yes.HAS_FIXED_VOICE_HH_PMIN4 == "1"),1).otherwise(0))
mob_09_tele_yes = mob_09_tele_yes.withColumn("HAS_FIXED_BB_HH_PMIN2",F.when((mob_09_tele_yes.HAS_FIXED_BB_HH_PMIN2 == "Y") | (mob_09_tele_yes.HAS_FIXED_BB_HH_PMIN2 == "1"),1).otherwise(0))
mob_09_tele_yes = mob_09_tele_yes.withColumn("HAS_FIXED_BB_HH_PMIN3",F.when((mob_09_tele_yes.HAS_FIXED_BB_HH_PMIN3 == "Y") | (mob_09_tele_yes.HAS_FIXED_BB_HH_PMIN3 == "1"),1).otherwise(0))
mob_09_tele_yes = mob_09_tele_yes.withColumn("HAS_FIXED_BB_HH_PMIN4",F.when((mob_09_tele_yes.HAS_FIXED_BB_HH_PMIN4 == "Y") | (mob_09_tele_yes.HAS_FIXED_BB_HH_PMIN4 == "1"),1).otherwise(0))
mob_09_tele_yes = mob_09_tele_yes.withColumn("HAS_FIXED_TV_HH_PMIN2",F.when((mob_09_tele_yes.HAS_FIXED_TV_HH_PMIN2 == "Y") | (mob_09_tele_yes.HAS_FIXED_TV_HH_PMIN2 == "1"),1).otherwise(0))
mob_09_tele_yes = mob_09_tele_yes.withColumn("HAS_FIXED_TV_HH_PMIN3",F.when((mob_09_tele_yes.HAS_FIXED_TV_HH_PMIN3 == "Y") | (mob_09_tele_yes.HAS_FIXED_TV_HH_PMIN3 == "1"),1).otherwise(0))
mob_09_tele_yes = mob_09_tele_yes.withColumn("HAS_FIXED_TV_HH_PMIN4",F.when((mob_09_tele_yes.HAS_FIXED_TV_HH_PMIN4 == "Y") | (mob_09_tele_yes.HAS_FIXED_TV_HH_PMIN4 == "1"),1).otherwise(0))
mob_09_tele_yes = mob_09_tele_yes.withColumn("FULL_TO_PARTIAL_MAGENTA_PMIN43",F.when((mob_09_tele_yes.FULL_TO_PARTIAL_MAGENTA_PMIN43 == "Y") | (mob_09_tele_yes.FULL_TO_PARTIAL_MAGENTA_PMIN43 == "1"),1).otherwise(0))
mob_09_tele_yes = mob_09_tele_yes.withColumn("FULL_TO_PARTIAL_MAGENTA_PMIN32",F.when((mob_09_tele_yes.FULL_TO_PARTIAL_MAGENTA_PMIN32 == "Y") | (mob_09_tele_yes.FULL_TO_PARTIAL_MAGENTA_PMIN32 == "1"),1).otherwise(0))
mob_09_tele_yes = mob_09_tele_yes.withColumn("FULL_TO_NO_MAGENTA_PMIN43",F.when((mob_09_tele_yes.FULL_TO_NO_MAGENTA_PMIN43 == "Y") | (mob_09_tele_yes.FULL_TO_NO_MAGENTA_PMIN43 == "1"),1).otherwise(0))
mob_09_tele_yes = mob_09_tele_yes.withColumn("FULL_TO_NO_MAGENTA_PMIN32",F.when((mob_09_tele_yes.FULL_TO_NO_MAGENTA_PMIN32 == "Y") | (mob_09_tele_yes.FULL_TO_NO_MAGENTA_PMIN32 == "1"),1).otherwise(0))
mob_09_tele_yes = mob_09_tele_yes.withColumn("PARTIAL_TO_NO_MAGENTA_PMIN43",F.when((mob_09_tele_yes.PARTIAL_TO_NO_MAGENTA_PMIN43 == "Y") | (mob_09_tele_yes.PARTIAL_TO_NO_MAGENTA_PMIN43 == "1"),1).otherwise(0))
mob_09_tele_yes = mob_09_tele_yes.withColumn("PARTIAL_TO_NO_MAGENTA_PMIN32",F.when((mob_09_tele_yes.PARTIAL_TO_NO_MAGENTA_PMIN32 == "Y") | (mob_09_tele_yes.PARTIAL_TO_NO_MAGENTA_PMIN32 == "1"),1).otherwise(0))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [150]:
# mob_09_tele_yes = mob_09_tele_yes.withColumn("DATA_ONLY_TARIFF",quotes("DATA_ONLY_TARIFF"))
# mob_09_tele_yes = mob_09_tele_yes.withColumn("HAS_FIXED_VOICE_HH_PMIN2",quotes("HAS_FIXED_VOICE_HH_PMIN2"))
# mob_09_tele_yes = mob_09_tele_yes.withColumn("HAS_FIXED_VOICE_HH_PMIN3",quotes("HAS_FIXED_VOICE_HH_PMIN3"))
# mob_09_tele_yes = mob_09_tele_yes.withColumn("HAS_FIXED_VOICE_HH_PMIN4",quotes("HAS_FIXED_VOICE_HH_PMIN4"))
# mob_09_tele_yes = mob_09_tele_yes.withColumn("HAS_FIXED_BB_HH_PMIN2",quotes("HAS_FIXED_BB_HH_PMIN2"))
# mob_09_tele_yes = mob_09_tele_yes.withColumn("HAS_FIXED_BB_HH_PMIN3",quotes("HAS_FIXED_BB_HH_PMIN3"))
# mob_09_tele_yes = mob_09_tele_yes.withColumn("HAS_FIXED_BB_HH_PMIN4",quotes("HAS_FIXED_BB_HH_PMIN4"))
# mob_09_tele_yes = mob_09_tele_yes.withColumn("HAS_FIXED_TV_HH_PMIN2",quotes("HAS_FIXED_TV_HH_PMIN2"))
# mob_09_tele_yes = mob_09_tele_yes.withColumn("HAS_FIXED_TV_HH_PMIN3",quotes("HAS_FIXED_TV_HH_PMIN3"))
# mob_09_tele_yes = mob_09_tele_yes.withColumn("HAS_FIXED_TV_HH_PMIN4",quotes("HAS_FIXED_TV_HH_PMIN4"))
# mob_09_tele_yes = mob_09_tele_yes.withColumn("FULL_TO_PARTIAL_MAGENTA_PMIN43",quotes("FULL_TO_PARTIAL_MAGENTA_PMIN43"))
# mob_09_tele_yes = mob_09_tele_yes.withColumn("FULL_TO_PARTIAL_MAGENTA_PMIN32",quotes("FULL_TO_PARTIAL_MAGENTA_PMIN32"))
# mob_09_tele_yes = mob_09_tele_yes.withColumn("FULL_TO_NO_MAGENTA_PMIN43",quotes("FULL_TO_NO_MAGENTA_PMIN43"))
# mob_09_tele_yes = mob_09_tele_yes.withColumn("FULL_TO_NO_MAGENTA_PMIN32",quotes("FULL_TO_NO_MAGENTA_PMIN32"))
# mob_09_tele_yes = mob_09_tele_yes.withColumn("PARTIAL_TO_NO_MAGENTA_PMIN43",quotes("PARTIAL_TO_NO_MAGENTA_PMIN43"))
# mob_09_tele_yes = mob_09_tele_yes.withColumn("PARTIAL_TO_NO_MAGENTA_PMIN32",quotes("PARTIAL_TO_NO_MAGENTA_PMIN32"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [96]:
mob_09_tele_yes = mob_09_tele_yes.groupBy("oib","calendar_date").agg(
    
    F.collect_list("DATA_ONLY_TARIFF").alias("DATA_ONLY_TARIFF"),
F.collect_list("HAS_FIXED_VOICE_HH_PMIN2").alias("HAS_FIXED_VOICE_HH_PMIN2"),
F.collect_list("HAS_FIXED_VOICE_HH_PMIN3").alias("HAS_FIXED_VOICE_HH_PMIN3"),
F.collect_list("HAS_FIXED_VOICE_HH_PMIN4").alias("HAS_FIXED_VOICE_HH_PMIN4"),
F.collect_list("HAS_FIXED_BB_HH_PMIN2").alias("HAS_FIXED_BB_HH_PMIN2"),
F.collect_list("HAS_FIXED_BB_HH_PMIN3").alias("HAS_FIXED_BB_HH_PMIN3"),
F.collect_list("HAS_FIXED_BB_HH_PMIN4").alias("HAS_FIXED_BB_HH_PMIN4"),
F.collect_list("HAS_FIXED_TV_HH_PMIN2").alias("HAS_FIXED_TV_HH_PMIN2"),
F.collect_list("HAS_FIXED_TV_HH_PMIN3").alias("HAS_FIXED_TV_HH_PMIN3"),
F.collect_list("HAS_FIXED_TV_HH_PMIN4").alias("HAS_FIXED_TV_HH_PMIN4"),
F.collect_list("FULL_TO_PARTIAL_MAGENTA_PMIN43").alias("FULL_TO_PARTIAL_MAGENTA_PMIN43"),
F.collect_list("FULL_TO_PARTIAL_MAGENTA_PMIN32").alias("FULL_TO_PARTIAL_MAGENTA_PMIN32"),
F.collect_list("FULL_TO_NO_MAGENTA_PMIN43").alias("FULL_TO_NO_MAGENTA_PMIN43"),
F.collect_list("FULL_TO_NO_MAGENTA_PMIN32").alias("FULL_TO_NO_MAGENTA_PMIN32"),
F.collect_list("PARTIAL_TO_NO_MAGENTA_PMIN43").alias("PARTIAL_TO_NO_MAGENTA_PMIN43"),
F.collect_list("PARTIAL_TO_NO_MAGENTA_PMIN32").alias("PARTIAL_TO_NO_MAGENTA_PMIN32")).withColumn("DATA_ONLY_TARIFF",YESorNO("DATA_ONLY_TARIFF")).withColumn("HAS_FIXED_VOICE_HH_PMIN2",YESorNO("HAS_FIXED_VOICE_HH_PMIN2")).withColumn("HAS_FIXED_VOICE_HH_PMIN3",YESorNO("HAS_FIXED_VOICE_HH_PMIN3")).withColumn("HAS_FIXED_VOICE_HH_PMIN4",YESorNO("HAS_FIXED_VOICE_HH_PMIN4")).withColumn("HAS_FIXED_BB_HH_PMIN2",YESorNO("HAS_FIXED_BB_HH_PMIN2")).withColumn("HAS_FIXED_BB_HH_PMIN3",YESorNO("HAS_FIXED_BB_HH_PMIN3")).withColumn("HAS_FIXED_BB_HH_PMIN4",YESorNO("HAS_FIXED_BB_HH_PMIN4")).withColumn("HAS_FIXED_TV_HH_PMIN2",YESorNO("HAS_FIXED_TV_HH_PMIN2")).withColumn("HAS_FIXED_TV_HH_PMIN3",YESorNO("HAS_FIXED_TV_HH_PMIN3")).withColumn("HAS_FIXED_TV_HH_PMIN4",YESorNO("HAS_FIXED_TV_HH_PMIN4")).withColumn("FULL_TO_PARTIAL_MAGENTA_PMIN43",YESorNO("FULL_TO_PARTIAL_MAGENTA_PMIN43")).withColumn("FULL_TO_PARTIAL_MAGENTA_PMIN32",YESorNO("FULL_TO_PARTIAL_MAGENTA_PMIN32")).withColumn("FULL_TO_NO_MAGENTA_PMIN43",YESorNO("FULL_TO_NO_MAGENTA_PMIN43")).withColumn("FULL_TO_NO_MAGENTA_PMIN32",YESorNO("FULL_TO_NO_MAGENTA_PMIN32")).withColumn("PARTIAL_TO_NO_MAGENTA_PMIN43",YESorNO("PARTIAL_TO_NO_MAGENTA_PMIN43")).withColumn("PARTIAL_TO_NO_MAGENTA_PMIN32",YESorNO("PARTIAL_TO_NO_MAGENTA_PMIN32"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [97]:
mob_09_tele_first.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

7318347

In [98]:
mob_09_tele_max.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

7318347

In [99]:
mob_09_tele_avg.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

7318347

In [100]:
mob_09_tele_yes.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

7318347

In [105]:
mob_09_tele_1 = mob_09_tele_max.join(mob_09_tele_first,["oib","calendar_date"],"left")
mob_09_tele_1 = mob_09_tele_1.join(mob_09_tele_avg,["oib","calendar_date"],"left")
mob_09_tele_1 = mob_09_tele_1.join(mob_09_tele_yes,["oib","calendar_date"],"left")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [112]:
mob_09_tele_1.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

7318347

In [107]:

# mob_09_tele_1.createOrReplaceTempView("myTemp11")
# sqlContext.sql("create table cdl_common.30march_Mob_after_agg_ as select * from myTemp11")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]

In [114]:
Fix_mob = Fix_final.join(mob_09_tele_1,(Fix_final.oib_kupac == mob_09_tele_1.oib)&(Fix_final.date  ==  mob_09_tele_1.calendar_date),"left")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [129]:
Fix_mob.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

733513

In [130]:
Fix_mob= Fix_mob.join(mob_09_tele_group,["oib"],"left")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [131]:
Fix_mob.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

733513

In [132]:

Fix_mob.createOrReplaceTempView("myTemp11")
sqlContext.sql("create table cdl_common.30march_Fix_new_1 as select * from myTemp11")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]